# IDP MD Simulation Analysis Protocol:
------
Developed for analysis of wildtype simulation 

## Notes:
#### Preprocessing
- Must specify protein length and window sizes
- Data is normalized to 0-1
---
#### Feature Enrichment
- No enrichment or filtering
---
#### Autoencoder Tuning
- No Tuning - Predetermined structure
---
#### Autoencoder Training
- Default Structure, Default Data
- 4, 5, 7, 8 Layer Models
- Local Compaction & Local Angle Data
- Unshuffled Data
---
#### Clustering
- Agglomerative Co-Clustering

In [1]:
import os
import re
import sys
import glob
import time
import json
import subprocess
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from scipy.stats import mode
from kneed import KneeLocator
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from IPython.display import Image, display

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Activation, Dense, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import Callback

2024-12-01 18:14:05.409908: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-01 18:14:05.501794: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-01 18:14:05.535464: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-01 18:14:05.546514: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 18:14:05.617518: I tensorflow/core/platform/cpu_feature_guar

# Data Import

In [2]:
import glob
import pandas as pd
import os
import re

# Define the range of window sizes
window_range = range(2, 52)

def import_lcc_data(lccdata_folder):
    """
    Imports Local Compaction data files (.csv) and assigns them to a dictionary.

    Parameters:
    - lccdata_folder: The folder where Local Compaction data files are stored.

    Returns:
    - A dictionary with window sizes as keys and pandas DataFrames as values.
    """
    data_dict = {}
    
    for window_size in window_range:
        file_path = os.path.join(lccdata_folder, f"WT_Simulation_WS_{window_size}.csv")
        if os.path.exists(file_path):
            df = pd.read_csv(file_path, index_col=0)
            df.index.name = None 
            data_dict[window_size] = df
        else:
            print(f"Warning: File not found for Window Size {window_size}: {file_path}")
    
    return data_dict


# Folder containing the Local Compaction data
lccdata_folder = 'Local_Compaction/Local_Compaction_Data'

# Import LCC data for the wild-type protein
wt_dict = import_lcc_data(lccdata_folder)

# Data Preparation for Dimension Reduction

### Data Preparation:
- Unfiltered, Shuffled Distance Measurement Data
- Unfiltered, Shuffled Angle Measurement Data
- Unfiltered, Shuffled RMSD Measurement Data
- Unfiltered, UnShuffled Distance Measurement Data
- Filtered, Shuffled Distance Measurement Data

## Unfiltered, Shuffled Distance Measurement Data, Normalized 0-1

In [3]:
import os
import pandas as pd
from sklearn.model_selection import KFold

def save_datasets(X_train, y_train, X_valid, y_valid, fold, base_folder):
    """
    Save the datasets for a specific fold.
    """
    folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
    os.makedirs(folder_name, exist_ok=True)
    
    X_train.to_csv(os.path.join(folder_name, 'X_train_f.csv'))
    y_train.to_csv(os.path.join(folder_name, 'y_train_f.csv'))
    X_valid.to_csv(os.path.join(folder_name, 'X_valid_f.csv'))
    y_valid.to_csv(os.path.join(folder_name, 'y_valid_f.csv'))

def load_datasets(fold, base_folder):
    """
    Load datasets for a specific fold.
    """
    folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
    
    X_train = pd.read_csv(os.path.join(folder_name, 'X_train_f.csv'), index_col=0)
    y_train = pd.read_csv(os.path.join(folder_name, 'y_train_f.csv'), index_col=0)
    X_valid = pd.read_csv(os.path.join(folder_name, 'X_valid_f.csv'), index_col=0)
    y_valid = pd.read_csv(os.path.join(folder_name, 'y_valid_f.csv'), index_col=0)
    
    return X_train, y_train, X_valid, y_valid

def normalize_to_range_0_1(data):
    """
    Normalize data to range [0, 1] and round to 6 decimal places.
    """
    normalized = (data - data.min()) / (data.max() - data.min())
    return normalized.round(6)

def preprocessing_kfold(wt_data, n_splits=5, base_folder="AE_Data/Unfiltered_Shuffled_Compaction_Data"):
    """
    Preprocess data using KFold cross-validation, normalize it, and save it.
    """
    # Ensure all data is numeric
    wt_data = wt_data.apply(pd.to_numeric, errors='coerce')

    # Shuffle the DataFrame rows without resetting the index
    wt_data = wt_data.sample(frac=1, random_state=42)
    
    os.makedirs(base_folder, exist_ok=True)

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold = 1
    for train_index, valid_index in kf.split(wt_data):
        folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
        if not os.path.exists(os.path.join(folder_name, 'X_train_f.csv')):
            X_train, X_valid = wt_data.iloc[train_index], wt_data.iloc[valid_index]
            y_train = pd.DataFrame({'class': [0] * len(X_train)}, index=X_train.index)
            y_valid = pd.DataFrame({'class': [0] * len(X_valid)}, index=X_valid.index)

            # Normalize training and validation data to [0, 1] and round to 6 decimal places
            X_train = normalize_to_range_0_1(X_train)
            X_valid = normalize_to_range_0_1(X_valid)

            print(f"Fold {fold}:")
            print(f"Training set shape: {X_train.shape}")
            print(f"Validation set shape: {X_valid.shape}")
            
            save_datasets(X_train, y_train, X_valid, y_valid, fold, base_folder)
        else:
            print(f"Data for fold {fold} already exists, skipping generation.")
        fold += 1

In [4]:
# Window sizes to be used
window_sizes = list(range(2, 52))

# Concatenate all DataFrames in wt_dict into a single DataFrame
wt_combined = pd.concat(wt_dict.values(), axis=1)

# Preprocess and save data
preprocessing_kfold(wt_combined, n_splits=5)

# Example of loading a specific fold (e.g., fold 1)
base_folder = "AE_Data/Unfiltered_Shuffled_Compaction_Data"
X_train_loaded, y_train_loaded, X_valid_loaded, y_valid_loaded = load_datasets(fold=1, base_folder=base_folder)

Data for fold 1 already exists, skipping generation.
Data for fold 2 already exists, skipping generation.
Data for fold 3 already exists, skipping generation.
Data for fold 4 already exists, skipping generation.
Data for fold 5 already exists, skipping generation.


## Unfiltered, UnShuffled Distance Measurement Data, Normalized 0-1

In [2]:
import os
import pandas as pd
from sklearn.model_selection import KFold

def save_datasets(X_train, y_train, X_valid, y_valid, fold, base_folder):
    """
    Save the datasets for a specific fold.
    """
    folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
    os.makedirs(folder_name, exist_ok=True)
    
    X_train.to_csv(os.path.join(folder_name, 'X_train_f.csv'))
    y_train.to_csv(os.path.join(folder_name, 'y_train_f.csv'))
    X_valid.to_csv(os.path.join(folder_name, 'X_valid_f.csv'))
    y_valid.to_csv(os.path.join(folder_name, 'y_valid_f.csv'))

def load_datasets(fold, base_folder):
    """
    Load datasets for a specific fold.
    """
    folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
    
    X_train = pd.read_csv(os.path.join(folder_name, 'X_train_f.csv'), index_col=0)
    y_train = pd.read_csv(os.path.join(folder_name, 'y_train_f.csv'), index_col=0)
    X_valid = pd.read_csv(os.path.join(folder_name, 'X_valid_f.csv'), index_col=0)
    y_valid = pd.read_csv(os.path.join(folder_name, 'y_valid_f.csv'), index_col=0)
    
    return X_train, y_train, X_valid, y_valid

def normalize_to_range_0_1(data):
    """
    Normalize data to range [0, 1] and round to 6 decimal places.
    """
    normalized = (data - data.min()) / (data.max() - data.min())
    return normalized.round(6)

def preprocessing_kfold(wt_data, n_splits=5, base_folder="AE_Data/Unfiltered_UnShuffled_Compaction_Data"):
    """
    Preprocess data using KFold cross-validation, normalize it, and save it without shuffling.
    """
    # Ensure all data is numeric
    wt_data = wt_data.apply(pd.to_numeric, errors='coerce')
    
    os.makedirs(base_folder, exist_ok=True)

    kf = KFold(n_splits=n_splits, shuffle=False)  # Do not shuffle
    fold = 1
    for train_index, valid_index in kf.split(wt_data):
        folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
        if not os.path.exists(os.path.join(folder_name, 'X_train_f.csv')):
            X_train, X_valid = wt_data.iloc[train_index], wt_data.iloc[valid_index]
            y_train = pd.DataFrame({'class': [0] * len(X_train)}, index=X_train.index)
            y_valid = pd.DataFrame({'class': [0] * len(X_valid)}, index=X_valid.index)

            # Normalize training and validation data to [0, 1] and round to 6 decimal places
            X_train = normalize_to_range_0_1(X_train)
            X_valid = normalize_to_range_0_1(X_valid)

            print(f"Fold {fold}:")
            print(f"Training set shape: {X_train.shape}")
            print(f"Validation set shape: {X_valid.shape}")
            
            save_datasets(X_train, y_train, X_valid, y_valid, fold, base_folder)
        else:
            print(f"Data for fold {fold} already exists, skipping generation.")
        fold += 1


In [20]:
# Window sizes to be used
window_sizes = list(range(2, 52))

# Concatenate all DataFrames in wt_dict into a single DataFrame
wt_combined = pd.concat(wt_dict.values(), axis=1)

# Preprocess and save data
preprocessing_kfold(wt_combined, n_splits=5)

# Example of loading a specific fold (e.g., fold 1)
base_folder = "AE_Data/Unfiltered_UnShuffled_Compaction_Data"
X_train_loaded, y_train_loaded, X_valid_loaded, y_valid_loaded = load_datasets(fold=1, base_folder=base_folder)

Fold 1:
Training set shape: (32000, 2175)
Validation set shape: (8000, 2175)
Fold 2:
Training set shape: (32000, 2175)
Validation set shape: (8000, 2175)
Fold 3:
Training set shape: (32000, 2175)
Validation set shape: (8000, 2175)
Fold 4:
Training set shape: (32000, 2175)
Validation set shape: (8000, 2175)
Fold 5:
Training set shape: (32000, 2175)
Validation set shape: (8000, 2175)


# Filtered, Shuffled Measurement Data, Normalized 0-1

In [1]:
import glob
import pandas as pd
import os
import re

# Define the range of window sizes
window_range = range(2, 49)

def import_lcc_data(lccdata_folder):
    """
    Imports Local Compaction data files (.csv) and assigns them to a dictionary.

    Parameters:
    - lccdata_folder: The folder where Local Compaction data files are stored.

    Returns:
    - A dictionary with window sizes as keys and pandas DataFrames as values.
    """
    data_dict = {}
    
    for window_size in window_range:
        file_path = os.path.join(lccdata_folder, f"WS_{window_size}_f.csv")
        if os.path.exists(file_path):
            df = pd.read_csv(file_path, index_col=0)
            df.index.name = None 
            data_dict[window_size] = df
        else:
            print(f"Warning: File not found for Window Size {window_size}: {file_path}")
    
    return data_dict


# Folder containing the Local Compaction data
lccdata_folder = 'XGB_High_vs_Low_Energy/Filtered_Local_Compaction_Data'

# Import LCC data for the wild-type protein
wt_dict = import_lcc_data(lccdata_folder)

In [2]:
import os
import pandas as pd
from sklearn.model_selection import KFold

def save_datasets(X_train, y_train, X_valid, y_valid, fold, base_folder):
    """
    Save the datasets for a specific fold.
    """
    folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
    os.makedirs(folder_name, exist_ok=True)
    
    X_train.to_csv(os.path.join(folder_name, 'X_train_f.csv'))
    y_train.to_csv(os.path.join(folder_name, 'y_train_f.csv'))
    X_valid.to_csv(os.path.join(folder_name, 'X_valid_f.csv'))
    y_valid.to_csv(os.path.join(folder_name, 'y_valid_f.csv'))

def load_datasets(fold, base_folder):
    """
    Load datasets for a specific fold.
    """
    folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
    
    X_train = pd.read_csv(os.path.join(folder_name, 'X_train_f.csv'), index_col=0)
    y_train = pd.read_csv(os.path.join(folder_name, 'y_train_f.csv'), index_col=0)
    X_valid = pd.read_csv(os.path.join(folder_name, 'X_valid_f.csv'), index_col=0)
    y_valid = pd.read_csv(os.path.join(folder_name, 'y_valid_f.csv'), index_col=0)
    
    return X_train, y_train, X_valid, y_valid

def normalize_to_range_0_1(data):
    """
    Normalize data to range [0, 1] and round to 6 decimal places.
    """
    normalized = (data - data.min()) / (data.max() - data.min())
    return normalized.round(6)

def preprocessing_kfold(wt_data, n_splits=5, base_folder="AE_Data/Filtered_Shuffled_Compaction_Data"):
    """
    Preprocess data using KFold cross-validation, normalize it, and save it.
    """
    # Ensure all data is numeric
    wt_data = wt_data.apply(pd.to_numeric, errors='coerce')

    # Shuffle the DataFrame rows without resetting the index
    wt_data = wt_data.sample(frac=1, random_state=42)
    
    os.makedirs(base_folder, exist_ok=True)

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold = 1
    for train_index, valid_index in kf.split(wt_data):
        folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
        if not os.path.exists(os.path.join(folder_name, 'X_train_f.csv')):
            X_train, X_valid = wt_data.iloc[train_index], wt_data.iloc[valid_index]
            y_train = pd.DataFrame({'class': [0] * len(X_train)}, index=X_train.index)
            y_valid = pd.DataFrame({'class': [0] * len(X_valid)}, index=X_valid.index)

            # Normalize training and validation data to [0, 1] and round to 6 decimal places
            X_train = normalize_to_range_0_1(X_train)
            X_valid = normalize_to_range_0_1(X_valid)

            print(f"Fold {fold}:")
            print(f"Training set shape: {X_train.shape}")
            print(f"Validation set shape: {X_valid.shape}")
            
            save_datasets(X_train, y_train, X_valid, y_valid, fold, base_folder)
        else:
            print(f"Data for fold {fold} already exists, skipping generation.")
        fold += 1

In [3]:
# Window sizes to be used
window_sizes = list(range(2, 52))

# Concatenate all DataFrames in wt_dict into a single DataFrame
wt_combined = pd.concat(wt_dict.values(), axis=1)

# Preprocess and save data
preprocessing_kfold(wt_combined, n_splits=5)

# Example of loading a specific fold (e.g., fold 1)
base_folder = "AE_Data/Filtered_Shuffled_Compaction_Data"
X_train_loaded, y_train_loaded, X_valid_loaded, y_valid_loaded = load_datasets(fold=1, base_folder=base_folder)

Data for fold 1 already exists, skipping generation.
Data for fold 2 already exists, skipping generation.
Data for fold 3 already exists, skipping generation.
Data for fold 4 already exists, skipping generation.
Data for fold 5 already exists, skipping generation.


# Unfiltered, Shuffled Angle Measurement Data, Normalized 0-1

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import KFold

def save_datasets(X_train, y_train, X_valid, y_valid, fold, base_folder):
    """
    Save the datasets for a specific fold.
    """
    folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
    os.makedirs(folder_name, exist_ok=True)
    
    X_train.to_csv(os.path.join(folder_name, 'X_train_f.csv'))
    y_train.to_csv(os.path.join(folder_name, 'y_train_f.csv'))
    X_valid.to_csv(os.path.join(folder_name, 'X_valid_f.csv'))
    y_valid.to_csv(os.path.join(folder_name, 'y_valid_f.csv'))

def load_datasets(fold, base_folder):
    """
    Load datasets for a specific fold.
    """
    folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
    
    X_train = pd.read_csv(os.path.join(folder_name, 'X_train_f.csv'), index_col=0)
    y_train = pd.read_csv(os.path.join(folder_name, 'y_train_f.csv'), index_col=0)
    X_valid = pd.read_csv(os.path.join(folder_name, 'X_valid_f.csv'), index_col=0)
    y_valid = pd.read_csv(os.path.join(folder_name, 'y_valid_f.csv'), index_col=0)
    
    return X_train, y_train, X_valid, y_valid

def normalize_to_range_0_1(data):
    """
    Normalize data to range [0, 1] and round to 6 decimal places.
    """
    normalized = (data - data.min()) / (data.max() - data.min())
    return normalized.round(6)

def load_angle_data(angle_folder, angle_window_sizes):
    """
    Load angle data for all window sizes and combine into a single DataFrame.
    """
    angle_data = {}
    for window_size in angle_window_sizes:
        file_path = os.path.join(angle_folder, f'Angles_WS_{window_size}.csv')
        if os.path.exists(file_path):
            angle_data[window_size] = pd.read_csv(file_path, index_col=0)
        else:
            print(f"Warning: Angle data file not found for window size {window_size}.")
    return angle_data

def combine_angle_data_only(angle_data, angle_window_sizes):
    """
    Combine all angle data into a single DataFrame, sorted by window size.
    """
    # Start with the index of one DataFrame (assuming all indices are consistent)
    first_window_size = list(angle_data.keys())[0]
    combined_data = pd.DataFrame(index=angle_data[first_window_size].index)

    for window_size in angle_window_sizes:
        if window_size in angle_data:
            combined_data = pd.concat([combined_data, angle_data[window_size]], axis=1)
        else:
            print(f"Warning: Missing angle data for window size {window_size}. Skipping.")
    
    return combined_data

def preprocessing_kfold(angle_data_only, n_splits=5, base_folder="AE_Data/Unfiltered_Shuffled_Angle_Data"):
    """
    Preprocess angle data using KFold cross-validation, normalize it, and save it.
    """
    # Ensure all data is numeric
    angle_data_only = angle_data_only.apply(pd.to_numeric, errors='coerce')

    # Shuffle the DataFrame rows without resetting the index
    angle_data_only = angle_data_only.sample(frac=1, random_state=42)
    
    os.makedirs(base_folder, exist_ok=True)

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold = 1
    for train_index, valid_index in kf.split(angle_data_only):
        folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
        if not os.path.exists(os.path.join(folder_name, 'X_train_f.csv')):
            X_train, X_valid = angle_data_only.iloc[train_index], angle_data_only.iloc[valid_index]
            y_train = pd.DataFrame({'class': [0] * len(X_train)}, index=X_train.index)
            y_valid = pd.DataFrame({'class': [0] * len(X_valid)}, index=X_valid.index)

            # Normalize training and validation data to [0, 1] and round to 6 decimal places
            X_train = normalize_to_range_0_1(X_train)
            X_valid = normalize_to_range_0_1(X_valid)

            print(f"Fold {fold}:")
            print(f"Training set shape: {X_train.shape}")
            print(f"Validation set shape: {X_valid.shape}")
            
            save_datasets(X_train, y_train, X_valid, y_valid, fold, base_folder)
        else:
            print(f"Data for fold {fold} already exists, skipping generation.")
        fold += 1

In [ ]:
# Paths and window sizes
angle_window_sizes = list(range(1, 51))
angle_folder = "Local_Angles/Angle_Data"

# Load and combine angle data only
angle_data = load_angle_data(angle_folder, angle_window_sizes)
combined_angle_data = combine_angle_data_only(angle_data, angle_window_sizes)

# Preprocess and save data
preprocessing_kfold(combined_angle_data, n_splits=5, base_folder="AE_Data/Unfiltered_Shuffled_Angle_Data")

# Example of loading a specific fold (e.g., fold 1)
base_folder = "AE_Data/Unfiltered_Shuffled_Angle_Data"
X_train_loaded, y_train_loaded, X_valid_loaded, y_valid_loaded = load_datasets(fold=1, base_folder=base_folder)

# Unfiltered, Shuffled RMSD Data, Normalized 0-1

# Unfiltered, Shuffled Rg Data, Normalized 0-1

## Unfiltered, Shuffled Distance and Angle Measurement Data, Normalized 0-1

In [26]:
import os
import pandas as pd
from sklearn.model_selection import KFold

def save_datasets(X_train, y_train, X_valid, y_valid, fold, base_folder):
    """
    Save the datasets for a specific fold.
    """
    folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
    os.makedirs(folder_name, exist_ok=True)
    
    X_train.to_csv(os.path.join(folder_name, 'X_train_f.csv'))
    y_train.to_csv(os.path.join(folder_name, 'y_train_f.csv'))
    X_valid.to_csv(os.path.join(folder_name, 'X_valid_f.csv'))
    y_valid.to_csv(os.path.join(folder_name, 'y_valid_f.csv'))

def load_datasets(fold, base_folder):
    """
    Load datasets for a specific fold.
    """
    folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
    
    X_train = pd.read_csv(os.path.join(folder_name, 'X_train_f.csv'), index_col=0)
    y_train = pd.read_csv(os.path.join(folder_name, 'y_train_f.csv'), index_col=0)
    X_valid = pd.read_csv(os.path.join(folder_name, 'X_valid_f.csv'), index_col=0)
    y_valid = pd.read_csv(os.path.join(folder_name, 'y_valid_f.csv'), index_col=0)
    
    return X_train, y_train, X_valid, y_valid

def normalize_to_range_0_1(data):
    """
    Normalize data to range [0, 1] and round to 6 decimal places.
    """
    normalized = (data - data.min()) / (data.max() - data.min())
    return normalized.round(6)

def load_angle_data(angle_folder, window_sizes):
    """
    Load angle data for all window sizes and combine into a single DataFrame.
    """
    angle_data = {}
    for window_size in window_sizes:
        file_path = os.path.join(angle_folder, f'Angles_WS_{window_size}.csv')
        if os.path.exists(file_path):
            angle_data[window_size] = pd.read_csv(file_path, index_col=0)
        else:
            print(f"Warning: Angle data file not found for window size {window_size}.")
    return angle_data

def combine_distance_and_angle(distance_data, angle_data, distance_window_sizes, angle_window_sizes):
    """
    Combine distance and angle data alternately by window size, starting with distance (window size 2).
    """
    # Start with the index of one DataFrame (assuming all indices are consistent)
    first_window_size = list(distance_data.keys())[0]
    combined_data = pd.DataFrame(index=distance_data[first_window_size].index)

    # Interleave distance and angle window sizes
    max_window_size = max(max(distance_window_sizes), max(angle_window_sizes))
    for i in range(1, max_window_size + 1):
        if i in distance_window_sizes and i in distance_data:
            combined_data = pd.concat([combined_data, distance_data[i]], axis=1)
        if i in angle_window_sizes and i in angle_data:
            combined_data = pd.concat([combined_data, angle_data[i]], axis=1)
    
    return combined_data



def preprocessing_kfold(combined_data, n_splits=5, base_folder="AE_Data/Unfiltered_Shuffled_Compaction_Angle_Data"):
    """
    Preprocess data using KFold cross-validation, normalize it, and save it.
    """
    # Ensure all data is numeric
    combined_data = combined_data.apply(pd.to_numeric, errors='coerce')

    # Shuffle the DataFrame rows without resetting the index
    combined_data = combined_data.sample(frac=1, random_state=42)
    
    os.makedirs(base_folder, exist_ok=True)

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold = 1
    for train_index, valid_index in kf.split(combined_data):
        folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
        if not os.path.exists(os.path.join(folder_name, 'X_train_f.csv')):
            X_train, X_valid = combined_data.iloc[train_index], combined_data.iloc[valid_index]
            y_train = pd.DataFrame({'class': [0] * len(X_train)}, index=X_train.index)
            y_valid = pd.DataFrame({'class': [0] * len(X_valid)}, index=X_valid.index)

            # Normalize training and validation data to [0, 1] and round to 6 decimal places
            X_train = normalize_to_range_0_1(X_train)
            X_valid = normalize_to_range_0_1(X_valid)

            print(f"Fold {fold}:")
            print(f"Training set shape: {X_train.shape}")
            print(f"Validation set shape: {X_valid.shape}")
            
            save_datasets(X_train, y_train, X_valid, y_valid, fold, base_folder)
        else:
            print(f"Data for fold {fold} already exists, skipping generation.")
        fold += 1

In [28]:
# Paths and window sizes
distance_window_sizes = list(range(2, 52))
angle_window_sizes = list(range(1, 51))

distance_data = {ws: pd.read_csv(f"Local_Compaction/Local_Compaction_Data/WT_Simulation_WS_{ws}.csv", index_col=0) for ws in distance_window_sizes}
angle_folder = "Local_Angles/Angle_Data"
angle_data = load_angle_data(angle_folder, angle_window_sizes)

# Combine distance and angle data alternately
combined_data = combine_distance_and_angle(distance_data, angle_data, distance_window_sizes, angle_window_sizes)

# Preprocess and save data
preprocessing_kfold(combined_data, n_splits=5, base_folder="AE_Data/Unfiltered_Shuffled_Compaction_Angle_Data")

# Example of loading a specific fold (e.g., fold 1)
base_folder = "AE_Data/Unfiltered_Shuffled_Compaction_Angle_Data"
X_train_loaded, y_train_loaded, X_valid_loaded, y_valid_loaded = load_datasets(fold=1, base_folder=base_folder)

Fold 1:
Training set shape: (32000, 4300)
Validation set shape: (8000, 4300)
Fold 2:
Training set shape: (32000, 4300)
Validation set shape: (8000, 4300)
Fold 3:
Training set shape: (32000, 4300)
Validation set shape: (8000, 4300)
Fold 4:
Training set shape: (32000, 4300)
Validation set shape: (8000, 4300)
Fold 5:
Training set shape: (32000, 4300)
Validation set shape: (8000, 4300)


# Autoencoder Training
----
Note: 
- Autoencoder trained on unfiltered, normalized local compaction data from 91-160C-Myc wildtype
- Each run has 5 autoencoders trained separately on the different exclusive test-train splits 

# Default AE Default Data (For Comparison)

In [5]:
# Save the model using the recommended Keras format
def save_model(model, model_path):
    if not model_path.endswith('.keras'):
        model_path += '.keras'
    model.save(model_path)
    
# Load the model
def load_existing_model(model_path):
    return load_model(model_path)

In [6]:
# Get autoencoder model
def AE_Model(train_data, LeReLU_alpha=0.01, batch_size=256):
    
    input_layer = Input(shape=(train_data.shape[1],), name='ae_input')
    
    encoder = Dense(1024, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e1')(input_layer)
    encoder = Dense(512, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e2')(encoder)
    encoder = Dense(256, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e3')(encoder)
    encoder = Dense(128, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e4')(encoder)
    encoder = Dense(64, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e5')(encoder)
    encoder = Dense(32, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e6')(encoder)

    encoded = Dense(2, activation='linear', name='ae_latent')(encoder)
    
    decoder = Dense(32, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d6')(encoded)
    decoder = Dense(64, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d5')(decoder)
    decoder = Dense(128, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d4')(decoder)
    decoder = Dense(256, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d3')(decoder)
    decoder = Dense(512, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d2')(decoder)
    decoder = Dense(1024, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d1')(decoder)

    output_layer = Dense(train_data.shape[1], activation='linear', name='ae_output')(decoder)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=1e-5), loss=MeanSquaredError())
    
    model.summary()
    
    return model

# Model 1

In [ ]:
base_folder = "AE_Data/Unfiltered_Shuffled_Compaction_Data"
X_train_loaded, y_train_loaded, X_valid_loaded, y_valid_loaded = load_datasets(fold=1, base_folder=base_folder)

In [10]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Define the output directory and subdirectories
output_dir = 'Default_AE_Default_Data/AE_Training_1'
subfolders = {
    'AE_Latent_Space_Data_TXT': os.path.join(output_dir, 'AE_Latent_Space_Data_TXT'),
    'AE_Training_Loss_Data': os.path.join(output_dir, 'AE_Training_Loss_Data'),
    'AE_Training_Loss_Graphs': os.path.join(output_dir, 'AE_Training_Loss_Graphs'),
    'AE_Latent_Space_Graphs': os.path.join(output_dir, 'AE_Latent_Space_Graphs'),
    'AE_Latent_Space_Data_PKL': os.path.join(output_dir, 'AE_Latent_Space_Data_PKL'),
    'models': os.path.join(output_dir, 'models'),
}

# Create all subdirectories
for folder in subfolders.values():
    os.makedirs(folder, exist_ok=True)

# Initialize the autoencoder with the loaded training data
autoencoder = AE_Model(X_train_loaded)

# Training loop
for counts in tqdm(range(101)):
    name = "Autoencoder_Training"

    # Save the latent space predictions log
    txt_file_path = os.path.join(subfolders['AE_Latent_Space_Data_TXT'], f"{counts}_Latent_Space_Predictions_Log.txt")
    with open(txt_file_path, "w") as file:
        # Train the model
        history = autoencoder.fit(
            X_train_loaded, X_train_loaded,
            epochs=1000,
            validation_data=(X_valid_loaded, X_valid_loaded),
            verbose=0
        )

        # Save training loss history
        training_history = pd.DataFrame(history.history)
        history_file_path = os.path.join(subfolders['AE_Training_Loss_Data'], f"{counts}_{name}_History.pkl")
        training_history.to_pickle(history_file_path)

        # Save training loss graph
        plt.plot(training_history)
        graph_file_path = os.path.join(subfolders['AE_Training_Loss_Graphs'], f"{counts}_{name}_History.png")
        plt.savefig(graph_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Define the latent space model
        dr_model = tf.keras.models.Model(
            inputs=autoencoder.input,
            outputs=autoencoder.get_layer('ae_latent').output
        )
        dr_model.summary(print_fn=lambda x: file.write(x + '\n'))

        # Initialize latent space variables
        x, y, z, indices = [], [], [], []

        # Process validation data in batches
        batch_size = 32
        for batch_start in range(0, len(X_valid_loaded), batch_size):
            batch_end = min(batch_start + batch_size, len(X_valid_loaded))
            X_batch = np.array(X_valid_loaded.iloc[batch_start:batch_end])
            y_batch = y_valid_loaded.iloc[batch_start:batch_end].values.flatten()

            # Predict latent space values
            op_batch = dr_model.predict(X_batch, verbose=0)
            for i, op in enumerate(op_batch):
                z.append(y_batch[i])
                x.append(op[0])
                y.append(op[1])
                indices.append(y_valid_loaded.index[batch_start + i])
                file.write(f"Prediction {batch_start + i}: {op}\n")

        # Save latent space scatter plot
        df = pd.DataFrame({'x': x, 'y': y, 'z': [f"trajectory-{k}" for k in z], 'index': indices})
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x='x', y='y', hue='z', data=df, s=10)
        scatter_file_path = os.path.join(subfolders['AE_Latent_Space_Graphs'], f"{counts}_Latent_Space.png")
        plt.savefig(scatter_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Save latent space data as pickle
        pkl_file_path = os.path.join(subfolders['AE_Latent_Space_Data_PKL'], f"{counts}_Latent_Space.pkl")
        df.to_pickle(pkl_file_path)

        # Save the model
        model_file_path = os.path.join(subfolders['models'], f"saved_model_{counts}.keras")
        save_model(autoencoder, model_file_path)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ ae_input (InputLayer)           │ (None, 2175)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e1 (Dense)                      │ (None, 1024)           │     2,228,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e2 (Dense)                      │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e3 (Dense)                      │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e4 (Dense)                      │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e5 (Dense)                      │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e6 (Dense)                      │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ae_latent (Dense)               │ (None, 2)              │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d6 (Dense)                      │ (None, 32)             │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d5 (Dense)                      │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d4 (Dense)                      │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d3 (Dense)                      │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d2 (Dense)                      │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d1 (Dense)                      │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ae_output (Dense)               │ (None, 2175)           │     2,229,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,857,473 (22.34 MB)

 Trainable params: 5,857,473 (22.34 MB)

 Non-trainable params: 0 (0.00 B)

...

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

# Model 2

In [ ]:
base_folder = "AE_Data/Unfiltered_Shuffled_Compaction_Data"
X_train_loaded, y_train_loaded, X_valid_loaded, y_valid_loaded = load_datasets(fold=2, base_folder=base_folder)

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Define the output directory and subdirectories
output_dir = 'Default_AE_Default_Data/AE_Training_2'
subfolders = {
    'AE_Latent_Space_Data_TXT': os.path.join(output_dir, 'AE_Latent_Space_Data_TXT'),
    'AE_Training_Loss_Data': os.path.join(output_dir, 'AE_Training_Loss_Data'),
    'AE_Training_Loss_Graphs': os.path.join(output_dir, 'AE_Training_Loss_Graphs'),
    'AE_Latent_Space_Graphs': os.path.join(output_dir, 'AE_Latent_Space_Graphs'),
    'AE_Latent_Space_Data_PKL': os.path.join(output_dir, 'AE_Latent_Space_Data_PKL'),
    'models': os.path.join(output_dir, 'models'),
}

# Create all subdirectories
for folder in subfolders.values():
    os.makedirs(folder, exist_ok=True)

# Initialize the autoencoder with the loaded training data
autoencoder = AE_Model(X_train_loaded)

# Training loop
for counts in tqdm(range(101)):
    name = "Autoencoder_Training"

    # Save the latent space predictions log
    txt_file_path = os.path.join(subfolders['AE_Latent_Space_Data_TXT'], f"{counts}_Latent_Space_Predictions_Log.txt")
    with open(txt_file_path, "w") as file:
        # Train the model
        history = autoencoder.fit(
            X_train_loaded, X_train_loaded,
            epochs=1000,
            validation_data=(X_valid_loaded, X_valid_loaded),
            verbose=0
        )

        # Save training loss history
        training_history = pd.DataFrame(history.history)
        history_file_path = os.path.join(subfolders['AE_Training_Loss_Data'], f"{counts}_{name}_History.pkl")
        training_history.to_pickle(history_file_path)

        # Save training loss graph
        plt.plot(training_history)
        graph_file_path = os.path.join(subfolders['AE_Training_Loss_Graphs'], f"{counts}_{name}_History.png")
        plt.savefig(graph_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Define the latent space model
        dr_model = tf.keras.models.Model(
            inputs=autoencoder.input,
            outputs=autoencoder.get_layer('ae_latent').output
        )
        dr_model.summary(print_fn=lambda x: file.write(x + '\n'))

        # Initialize latent space variables
        x, y, z, indices = [], [], [], []

        # Process validation data in batches
        batch_size = 32
        for batch_start in range(0, len(X_valid_loaded), batch_size):
            batch_end = min(batch_start + batch_size, len(X_valid_loaded))
            X_batch = np.array(X_valid_loaded.iloc[batch_start:batch_end])
            y_batch = y_valid_loaded.iloc[batch_start:batch_end].values.flatten()

            # Predict latent space values
            op_batch = dr_model.predict(X_batch, verbose=0)
            for i, op in enumerate(op_batch):
                z.append(y_batch[i])
                x.append(op[0])
                y.append(op[1])
                indices.append(y_valid_loaded.index[batch_start + i])
                file.write(f"Prediction {batch_start + i}: {op}\n")

        # Save latent space scatter plot
        df = pd.DataFrame({'x': x, 'y': y, 'z': [f"trajectory-{k}" for k in z], 'index': indices})
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x='x', y='y', hue='z', data=df, s=10)
        scatter_file_path = os.path.join(subfolders['AE_Latent_Space_Graphs'], f"{counts}_Latent_Space.png")
        plt.savefig(scatter_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Save latent space data as pickle
        pkl_file_path = os.path.join(subfolders['AE_Latent_Space_Data_PKL'], f"{counts}_Latent_Space.pkl")
        df.to_pickle(pkl_file_path)

        # Save the model
        model_file_path = os.path.join(subfolders['models'], f"saved_model_{counts}.keras")
        save_model(autoencoder, model_file_path)

# Model 3

In [ ]:
base_folder = "AE_Data/Unfiltered_Shuffled_Compaction_Data"
X_train_loaded, y_train_loaded, X_valid_loaded, y_valid_loaded = load_datasets(fold=3, base_folder=base_folder)

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Define the output directory and subdirectories
output_dir = 'Default_AE_Default_Data/AE_Training_3'
subfolders = {
    'AE_Latent_Space_Data_TXT': os.path.join(output_dir, 'AE_Latent_Space_Data_TXT'),
    'AE_Training_Loss_Data': os.path.join(output_dir, 'AE_Training_Loss_Data'),
    'AE_Training_Loss_Graphs': os.path.join(output_dir, 'AE_Training_Loss_Graphs'),
    'AE_Latent_Space_Graphs': os.path.join(output_dir, 'AE_Latent_Space_Graphs'),
    'AE_Latent_Space_Data_PKL': os.path.join(output_dir, 'AE_Latent_Space_Data_PKL'),
    'models': os.path.join(output_dir, 'models'),
}

# Create all subdirectories
for folder in subfolders.values():
    os.makedirs(folder, exist_ok=True)

# Initialize the autoencoder with the loaded training data
autoencoder = AE_Model(X_train_loaded)

# Training loop
for counts in tqdm(range(101)):
    name = "Autoencoder_Training"

    # Save the latent space predictions log
    txt_file_path = os.path.join(subfolders['AE_Latent_Space_Data_TXT'], f"{counts}_Latent_Space_Predictions_Log.txt")
    with open(txt_file_path, "w") as file:
        # Train the model
        history = autoencoder.fit(
            X_train_loaded, X_train_loaded,
            epochs=1000,
            validation_data=(X_valid_loaded, X_valid_loaded),
            verbose=0
        )

        # Save training loss history
        training_history = pd.DataFrame(history.history)
        history_file_path = os.path.join(subfolders['AE_Training_Loss_Data'], f"{counts}_{name}_History.pkl")
        training_history.to_pickle(history_file_path)

        # Save training loss graph
        plt.plot(training_history)
        graph_file_path = os.path.join(subfolders['AE_Training_Loss_Graphs'], f"{counts}_{name}_History.png")
        plt.savefig(graph_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Define the latent space model
        dr_model = tf.keras.models.Model(
            inputs=autoencoder.input,
            outputs=autoencoder.get_layer('ae_latent').output
        )
        dr_model.summary(print_fn=lambda x: file.write(x + '\n'))

        # Initialize latent space variables
        x, y, z, indices = [], [], [], []

        # Process validation data in batches
        batch_size = 32
        for batch_start in range(0, len(X_valid_loaded), batch_size):
            batch_end = min(batch_start + batch_size, len(X_valid_loaded))
            X_batch = np.array(X_valid_loaded.iloc[batch_start:batch_end])
            y_batch = y_valid_loaded.iloc[batch_start:batch_end].values.flatten()

            # Predict latent space values
            op_batch = dr_model.predict(X_batch, verbose=0)
            for i, op in enumerate(op_batch):
                z.append(y_batch[i])
                x.append(op[0])
                y.append(op[1])
                indices.append(y_valid_loaded.index[batch_start + i])
                file.write(f"Prediction {batch_start + i}: {op}\n")

        # Save latent space scatter plot
        df = pd.DataFrame({'x': x, 'y': y, 'z': [f"trajectory-{k}" for k in z], 'index': indices})
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x='x', y='y', hue='z', data=df, s=10)
        scatter_file_path = os.path.join(subfolders['AE_Latent_Space_Graphs'], f"{counts}_Latent_Space.png")
        plt.savefig(scatter_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Save latent space data as pickle
        pkl_file_path = os.path.join(subfolders['AE_Latent_Space_Data_PKL'], f"{counts}_Latent_Space.pkl")
        df.to_pickle(pkl_file_path)

        # Save the model
        model_file_path = os.path.join(subfolders['models'], f"saved_model_{counts}.keras")
        save_model(autoencoder, model_file_path)

# Model 4

In [ ]:
base_folder = "AE_Data/Unfiltered_Shuffled_Compaction_Data"
X_train_loaded, y_train_loaded, X_valid_loaded, y_valid_loaded = load_datasets(fold=4, base_folder=base_folder)

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Define the output directory and subdirectories
output_dir = 'Default_AE_Default_Data/AE_Training_4'
subfolders = {
    'AE_Latent_Space_Data_TXT': os.path.join(output_dir, 'AE_Latent_Space_Data_TXT'),
    'AE_Training_Loss_Data': os.path.join(output_dir, 'AE_Training_Loss_Data'),
    'AE_Training_Loss_Graphs': os.path.join(output_dir, 'AE_Training_Loss_Graphs'),
    'AE_Latent_Space_Graphs': os.path.join(output_dir, 'AE_Latent_Space_Graphs'),
    'AE_Latent_Space_Data_PKL': os.path.join(output_dir, 'AE_Latent_Space_Data_PKL'),
    'models': os.path.join(output_dir, 'models'),
}

# Create all subdirectories
for folder in subfolders.values():
    os.makedirs(folder, exist_ok=True)

# Initialize the autoencoder with the loaded training data
autoencoder = AE_Model(X_train_loaded)

# Training loop
for counts in tqdm(range(101)):
    name = "Autoencoder_Training"

    # Save the latent space predictions log
    txt_file_path = os.path.join(subfolders['AE_Latent_Space_Data_TXT'], f"{counts}_Latent_Space_Predictions_Log.txt")
    with open(txt_file_path, "w") as file:
        # Train the model
        history = autoencoder.fit(
            X_train_loaded, X_train_loaded,
            epochs=1000,
            validation_data=(X_valid_loaded, X_valid_loaded),
            verbose=0
        )

        # Save training loss history
        training_history = pd.DataFrame(history.history)
        history_file_path = os.path.join(subfolders['AE_Training_Loss_Data'], f"{counts}_{name}_History.pkl")
        training_history.to_pickle(history_file_path)

        # Save training loss graph
        plt.plot(training_history)
        graph_file_path = os.path.join(subfolders['AE_Training_Loss_Graphs'], f"{counts}_{name}_History.png")
        plt.savefig(graph_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Define the latent space model
        dr_model = tf.keras.models.Model(
            inputs=autoencoder.input,
            outputs=autoencoder.get_layer('ae_latent').output
        )
        dr_model.summary(print_fn=lambda x: file.write(x + '\n'))

        # Initialize latent space variables
        x, y, z, indices = [], [], [], []

        # Process validation data in batches
        batch_size = 32
        for batch_start in range(0, len(X_valid_loaded), batch_size):
            batch_end = min(batch_start + batch_size, len(X_valid_loaded))
            X_batch = np.array(X_valid_loaded.iloc[batch_start:batch_end])
            y_batch = y_valid_loaded.iloc[batch_start:batch_end].values.flatten()

            # Predict latent space values
            op_batch = dr_model.predict(X_batch, verbose=0)
            for i, op in enumerate(op_batch):
                z.append(y_batch[i])
                x.append(op[0])
                y.append(op[1])
                indices.append(y_valid_loaded.index[batch_start + i])
                file.write(f"Prediction {batch_start + i}: {op}\n")

        # Save latent space scatter plot
        df = pd.DataFrame({'x': x, 'y': y, 'z': [f"trajectory-{k}" for k in z], 'index': indices})
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x='x', y='y', hue='z', data=df, s=10)
        scatter_file_path = os.path.join(subfolders['AE_Latent_Space_Graphs'], f"{counts}_Latent_Space.png")
        plt.savefig(scatter_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Save latent space data as pickle
        pkl_file_path = os.path.join(subfolders['AE_Latent_Space_Data_PKL'], f"{counts}_Latent_Space.pkl")
        df.to_pickle(pkl_file_path)

        # Save the model
        model_file_path = os.path.join(subfolders['models'], f"saved_model_{counts}.keras")
        save_model(autoencoder, model_file_path)

# Model 5

In [ ]:
base_folder = "AE_Data/Unfiltered_Shuffled_Compaction_Data"
X_train_loaded, y_train_loaded, X_valid_loaded, y_valid_loaded = load_datasets(fold=5, base_folder=base_folder)

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Define the output directory and subdirectories
output_dir = 'Default_AE_Default_Data/AE_Training_5'
subfolders = {
    'AE_Latent_Space_Data_TXT': os.path.join(output_dir, 'AE_Latent_Space_Data_TXT'),
    'AE_Training_Loss_Data': os.path.join(output_dir, 'AE_Training_Loss_Data'),
    'AE_Training_Loss_Graphs': os.path.join(output_dir, 'AE_Training_Loss_Graphs'),
    'AE_Latent_Space_Graphs': os.path.join(output_dir, 'AE_Latent_Space_Graphs'),
    'AE_Latent_Space_Data_PKL': os.path.join(output_dir, 'AE_Latent_Space_Data_PKL'),
    'models': os.path.join(output_dir, 'models'),
}

# Create all subdirectories
for folder in subfolders.values():
    os.makedirs(folder, exist_ok=True)

# Initialize the autoencoder with the loaded training data
autoencoder = AE_Model(X_train_loaded)

# Training loop
for counts in tqdm(range(101)):
    name = "Autoencoder_Training"

    # Save the latent space predictions log
    txt_file_path = os.path.join(subfolders['AE_Latent_Space_Data_TXT'], f"{counts}_Latent_Space_Predictions_Log.txt")
    with open(txt_file_path, "w") as file:
        # Train the model
        history = autoencoder.fit(
            X_train_loaded, X_train_loaded,
            epochs=1000,
            validation_data=(X_valid_loaded, X_valid_loaded),
            verbose=0
        )

        # Save training loss history
        training_history = pd.DataFrame(history.history)
        history_file_path = os.path.join(subfolders['AE_Training_Loss_Data'], f"{counts}_{name}_History.pkl")
        training_history.to_pickle(history_file_path)

        # Save training loss graph
        plt.plot(training_history)
        graph_file_path = os.path.join(subfolders['AE_Training_Loss_Graphs'], f"{counts}_{name}_History.png")
        plt.savefig(graph_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Define the latent space model
        dr_model = tf.keras.models.Model(
            inputs=autoencoder.input,
            outputs=autoencoder.get_layer('ae_latent').output
        )
        dr_model.summary(print_fn=lambda x: file.write(x + '\n'))

        # Initialize latent space variables
        x, y, z, indices = [], [], [], []

        # Process validation data in batches
        batch_size = 32
        for batch_start in range(0, len(X_valid_loaded), batch_size):
            batch_end = min(batch_start + batch_size, len(X_valid_loaded))
            X_batch = np.array(X_valid_loaded.iloc[batch_start:batch_end])
            y_batch = y_valid_loaded.iloc[batch_start:batch_end].values.flatten()

            # Predict latent space values
            op_batch = dr_model.predict(X_batch, verbose=0)
            for i, op in enumerate(op_batch):
                z.append(y_batch[i])
                x.append(op[0])
                y.append(op[1])
                indices.append(y_valid_loaded.index[batch_start + i])
                file.write(f"Prediction {batch_start + i}: {op}\n")

        # Save latent space scatter plot
        df = pd.DataFrame({'x': x, 'y': y, 'z': [f"trajectory-{k}" for k in z], 'index': indices})
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x='x', y='y', hue='z', data=df, s=10)
        scatter_file_path = os.path.join(subfolders['AE_Latent_Space_Graphs'], f"{counts}_Latent_Space.png")
        plt.savefig(scatter_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Save latent space data as pickle
        pkl_file_path = os.path.join(subfolders['AE_Latent_Space_Data_PKL'], f"{counts}_Latent_Space.pkl")
        df.to_pickle(pkl_file_path)

        # Save the model
        model_file_path = os.path.join(subfolders['models'], f"saved_model_{counts}.keras")
        save_model(autoencoder, model_file_path)

# UnShuffled Raw Distance Data

In [3]:
# Save the model using the recommended Keras format
def save_model(model, model_path):
    if not model_path.endswith('.keras'):
        model_path += '.keras'
    model.save(model_path)
    
# Load the model
def load_existing_model(model_path):
    return load_model(model_path)

In [4]:
# Get autoencoder model
def AE_Model(train_data, LeReLU_alpha=0.01, batch_size=256):
    
    input_layer = Input(shape=(train_data.shape[1],), name='ae_input')
    
    encoder = Dense(1024, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e1')(input_layer)
    encoder = Dense(512, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e2')(encoder)
    encoder = Dense(256, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e3')(encoder)
    encoder = Dense(128, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e4')(encoder)
    encoder = Dense(64, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e5')(encoder)
    encoder = Dense(32, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e6')(encoder)

    encoded = Dense(2, activation='linear', name='ae_latent')(encoder)
    
    decoder = Dense(32, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d6')(encoded)
    decoder = Dense(64, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d5')(decoder)
    decoder = Dense(128, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d4')(decoder)
    decoder = Dense(256, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d3')(decoder)
    decoder = Dense(512, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d2')(decoder)
    decoder = Dense(1024, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d1')(decoder)

    output_layer = Dense(train_data.shape[1], activation='linear', name='ae_output')(decoder)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=1e-5), loss=MeanSquaredError())
    
    model.summary()
    
    return model

In [5]:
base_folder = "AE_Data/Unfiltered_UnShuffled_Compaction_Data"
X_train_loaded, y_train_loaded, X_valid_loaded, y_valid_loaded = load_datasets(fold=1, base_folder=base_folder)

In [7]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Define the output directory and subdirectories
output_dir = 'Default_AE_UnShuffled_Data/AE_Training_1'
subfolders = {
    'AE_Latent_Space_Data_TXT': os.path.join(output_dir, 'AE_Latent_Space_Data_TXT'),
    'AE_Training_Loss_Data': os.path.join(output_dir, 'AE_Training_Loss_Data'),
    'AE_Training_Loss_Graphs': os.path.join(output_dir, 'AE_Training_Loss_Graphs'),
    'AE_Latent_Space_Graphs': os.path.join(output_dir, 'AE_Latent_Space_Graphs'),
    'AE_Latent_Space_Data_PKL': os.path.join(output_dir, 'AE_Latent_Space_Data_PKL'),
    'models': os.path.join(output_dir, 'models'),
}

# Create all subdirectories
for folder in subfolders.values():
    os.makedirs(folder, exist_ok=True)

# Initialize the autoencoder with the loaded training data
autoencoder = AE_Model(X_train_loaded)

# Training loop
for counts in tqdm(range(101)):
    name = "Autoencoder_Training"

    # Save the latent space predictions log
    txt_file_path = os.path.join(subfolders['AE_Latent_Space_Data_TXT'], f"{counts}_Latent_Space_Predictions_Log.txt")
    with open(txt_file_path, "w") as file:
        # Train the model
        history = autoencoder.fit(
            X_train_loaded, X_train_loaded,
            epochs=1000,
            validation_data=(X_valid_loaded, X_valid_loaded),
            verbose=0
        )

        # Save training loss history
        training_history = pd.DataFrame(history.history)
        history_file_path = os.path.join(subfolders['AE_Training_Loss_Data'], f"{counts}_{name}_History.pkl")
        training_history.to_pickle(history_file_path)

        # Save training loss graph
        plt.plot(training_history)
        graph_file_path = os.path.join(subfolders['AE_Training_Loss_Graphs'], f"{counts}_{name}_History.png")
        plt.savefig(graph_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Define the latent space model
        dr_model = tf.keras.models.Model(
            inputs=autoencoder.input,
            outputs=autoencoder.get_layer('ae_latent').output
        )
        dr_model.summary(print_fn=lambda x: file.write(x + '\n'))

        # Initialize latent space variables
        x, y, z, indices = [], [], [], []

        # Process validation data in batches
        batch_size = 32
        for batch_start in range(0, len(X_valid_loaded), batch_size):
            batch_end = min(batch_start + batch_size, len(X_valid_loaded))
            X_batch = np.array(X_valid_loaded.iloc[batch_start:batch_end])
            y_batch = y_valid_loaded.iloc[batch_start:batch_end].values.flatten()

            # Predict latent space values
            op_batch = dr_model.predict(X_batch, verbose=0)
            for i, op in enumerate(op_batch):
                z.append(y_batch[i])
                x.append(op[0])
                y.append(op[1])
                indices.append(y_valid_loaded.index[batch_start + i])
                file.write(f"Prediction {batch_start + i}: {op}\n")

        # Save latent space scatter plot
        df = pd.DataFrame({'x': x, 'y': y, 'z': [f"trajectory-{k}" for k in z], 'index': indices})
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x='x', y='y', hue='z', data=df, s=10)
        scatter_file_path = os.path.join(subfolders['AE_Latent_Space_Graphs'], f"{counts}_Latent_Space.png")
        plt.savefig(scatter_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Save latent space data as pickle
        pkl_file_path = os.path.join(subfolders['AE_Latent_Space_Data_PKL'], f"{counts}_Latent_Space.pkl")
        df.to_pickle(pkl_file_path)

        # Save the model
        model_file_path = os.path.join(subfolders['models'], f"saved_model_{counts}.keras")
        save_model(autoencoder, model_file_path)

I0000 00:00:1732886411.338201    3919 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732886411.412068    3919 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732886411.412208    3919 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732886411.413644    3919 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ ae_input (InputLayer)           │ (None, 2175)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e1 (Dense)                      │ (None, 1024)           │     2,228,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e2 (Dense)                      │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e3 (Dense)                      │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e4 (Dense)                      │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e5 (Dense)                      │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e6 (Dense)                      │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ae_latent (Dense)               │ (None, 2)              │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d6 (Dense)                      │ (None, 32)             │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d5 (Dense)                      │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d4 (Dense)                      │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d3 (Dense)                      │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d2 (Dense)                      │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d1 (Dense)                      │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ae_output (Dense)               │ (None, 2175)           │     2,229,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,857,473 (22.34 MB)

 Trainable params: 5,857,473 (22.34 MB)

...

 96%|████████████████████████████████████▍ | 97/101 [20:52:01<51:17, 769.36s/it]

 97%|████████████████████████████████████▊ | 98/101 [21:04:49<38:26, 768.93s/it]

 98%|█████████████████████████████████████▏| 99/101 [21:17:38<25:38, 769.11s/it]

 99%|████████████████████████████████████▋| 100/101 [21:30:29<12:49, 769.45s/it]

100%|█████████████████████████████████████| 101/101 [21:43:17<00:00, 774.23s/it]


# Batch Size 512

# Model 1

In [5]:
# Save the model using the recommended Keras format
def save_model(model, model_path):
    if not model_path.endswith('.keras'):
        model_path += '.keras'
    model.save(model_path)
    
# Load the model
def load_existing_model(model_path):
    return load_model(model_path)

In [6]:
# Get autoencoder model
def AE_Model(train_data, LeReLU_alpha=0.01, batch_size=512):
    
    input_layer = Input(shape=(train_data.shape[1],), name='ae_input')
    
    encoder = Dense(1024, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e1')(input_layer)
    encoder = Dense(512, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e2')(encoder)
    encoder = Dense(256, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e3')(encoder)
    encoder = Dense(128, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e4')(encoder)
    encoder = Dense(64, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e5')(encoder)
    encoder = Dense(32, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e6')(encoder)

    encoded = Dense(2, activation='linear', name='ae_latent')(encoder)
    
    decoder = Dense(32, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d6')(encoded)
    decoder = Dense(64, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d5')(decoder)
    decoder = Dense(128, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d4')(decoder)
    decoder = Dense(256, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d3')(decoder)
    decoder = Dense(512, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d2')(decoder)
    decoder = Dense(1024, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d1')(decoder)

    output_layer = Dense(train_data.shape[1], activation='linear', name='ae_output')(decoder)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=1e-5), loss=MeanSquaredError())
    
    model.summary()
    
    return model

In [7]:
base_folder = "AE_Data/Unfiltered_Shuffled_Compaction_Data"
X_train_loaded, y_train_loaded, X_valid_loaded, y_valid_loaded = load_datasets(fold=1, base_folder=base_folder)

In [9]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Define the output directory and subdirectories
output_dir = 'Default_AE_BS_512_Default_Data/AE_Training_1'
subfolders = {
    'AE_Latent_Space_Data_TXT': os.path.join(output_dir, 'AE_Latent_Space_Data_TXT'),
    'AE_Training_Loss_Data': os.path.join(output_dir, 'AE_Training_Loss_Data'),
    'AE_Training_Loss_Graphs': os.path.join(output_dir, 'AE_Training_Loss_Graphs'),
    'AE_Latent_Space_Graphs': os.path.join(output_dir, 'AE_Latent_Space_Graphs'),
    'AE_Latent_Space_Data_PKL': os.path.join(output_dir, 'AE_Latent_Space_Data_PKL'),
    'models': os.path.join(output_dir, 'models'),
}

# Create all subdirectories
for folder in subfolders.values():
    os.makedirs(folder, exist_ok=True)

# Initialize the autoencoder with the loaded training data
autoencoder = AE_Model(X_train_loaded)

# Training loop
for counts in tqdm(range(101)):
    name = "Autoencoder_Training"

    # Save the latent space predictions log
    txt_file_path = os.path.join(subfolders['AE_Latent_Space_Data_TXT'], f"{counts}_Latent_Space_Predictions_Log.txt")
    with open(txt_file_path, "w") as file:
        # Train the model
        history = autoencoder.fit(
            X_train_loaded, X_train_loaded,
            epochs=1000,
            validation_data=(X_valid_loaded, X_valid_loaded),
            verbose=0
        )

        # Save training loss history
        training_history = pd.DataFrame(history.history)
        history_file_path = os.path.join(subfolders['AE_Training_Loss_Data'], f"{counts}_{name}_History.pkl")
        training_history.to_pickle(history_file_path)

        # Save training loss graph
        plt.plot(training_history)
        graph_file_path = os.path.join(subfolders['AE_Training_Loss_Graphs'], f"{counts}_{name}_History.png")
        plt.savefig(graph_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Define the latent space model
        dr_model = tf.keras.models.Model(
            inputs=autoencoder.input,
            outputs=autoencoder.get_layer('ae_latent').output
        )
        dr_model.summary(print_fn=lambda x: file.write(x + '\n'))

        # Initialize latent space variables
        x, y, z, indices = [], [], [], []

        # Process validation data in batches
        batch_size = 32
        for batch_start in range(0, len(X_valid_loaded), batch_size):
            batch_end = min(batch_start + batch_size, len(X_valid_loaded))
            X_batch = np.array(X_valid_loaded.iloc[batch_start:batch_end])
            y_batch = y_valid_loaded.iloc[batch_start:batch_end].values.flatten()

            # Predict latent space values
            op_batch = dr_model.predict(X_batch, verbose=0)
            for i, op in enumerate(op_batch):
                z.append(y_batch[i])
                x.append(op[0])
                y.append(op[1])
                indices.append(y_valid_loaded.index[batch_start + i])
                file.write(f"Prediction {batch_start + i}: {op}\n")

        # Save latent space scatter plot
        df = pd.DataFrame({'x': x, 'y': y, 'z': [f"trajectory-{k}" for k in z], 'index': indices})
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x='x', y='y', hue='z', data=df, s=10)
        scatter_file_path = os.path.join(subfolders['AE_Latent_Space_Graphs'], f"{counts}_Latent_Space.png")
        plt.savefig(scatter_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Save latent space data as pickle
        pkl_file_path = os.path.join(subfolders['AE_Latent_Space_Data_PKL'], f"{counts}_Latent_Space.pkl")
        df.to_pickle(pkl_file_path)

        # Save the model
        model_file_path = os.path.join(subfolders['models'], f"saved_model_{counts}.keras")
        save_model(autoencoder, model_file_path)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ ae_input (InputLayer)           │ (None, 2175)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e1 (Dense)                      │ (None, 1024)           │     2,228,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e2 (Dense)                      │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e3 (Dense)                      │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e4 (Dense)                      │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e5 (Dense)                      │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e6 (Dense)                      │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ae_latent (Dense)               │ (None, 2)              │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d6 (Dense)                      │ (None, 32)             │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d5 (Dense)                      │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d4 (Dense)                      │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d3 (Dense)                      │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d2 (Dense)                      │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d1 (Dense)                      │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ae_output (Dense)               │ (None, 2175)           │     2,229,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,857,473 (22.34 MB)

 Trainable params: 5,857,473 (22.34 MB)

 Non-trainable params: 0 (0.00 B)

...

 96%|████████████████████████████████████▍ | 97/101 [20:57:32<51:06, 766.73s/it]

 97%|████████████████████████████████████▊ | 98/101 [21:10:20<38:20, 766.89s/it]

 98%|█████████████████████████████████████▏| 99/101 [21:23:06<25:33, 766.83s/it]

 99%|████████████████████████████████████▋| 100/101 [21:35:54<12:47, 767.23s/it]

100%|█████████████████████████████████████| 101/101 [21:48:43<00:00, 777.46s/it]


# 8 Layer AE

# Model 1

In [ ]:
# Save the model using the recommended Keras format
def save_model(model, model_path):
    if not model_path.endswith('.keras'):
        model_path += '.keras'
    model.save(model_path)
    
# Load the model
def load_existing_model(model_path):
    return load_model(model_path)

In [ ]:
# Get autoencoder model
def AE_Model(train_data, LeReLU_alpha=0.01, batch_size=256):
    
    input_layer = Input(shape=(train_data.shape[1],), name='ae_input')

    encoder = Dense(4096, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e1')(input_layer)
    encoder = Dense(2048, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e2')(encoder)
    encoder = Dense(1024, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e3')(encoder)
    encoder = Dense(512, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e4')(encoder)
    encoder = Dense(256, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e5')(encoder)
    encoder = Dense(128, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e6')(encoder)
    encoder = Dense(64, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e7')(encoder)
    encoder = Dense(32, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e8')(encoder)

    encoded = Dense(2, activation='linear', name='ae_latent')(encoder)
    
    decoder = Dense(32, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d8')(encoded)
    decoder = Dense(64, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d7')(decoder)
    decoder = Dense(128, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d6')(decoder)
    decoder = Dense(256, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d5')(decoder)
    decoder = Dense(512, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d4')(decoder)
    decoder = Dense(1024, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d3')(decoder)
    decoder = Dense(2048, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d2')(decoder)
    decoder = Dense(4096, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d1')(decoder)
    

    output_layer = Dense(train_data.shape[1], activation='linear', name='ae_output')(decoder)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=1e-5), loss=MeanSquaredError())
    
    model.summary()
    
    return model

In [ ]:
def load_datasets(fold, base_folder):
    """
    Load datasets for a specific fold.
    """
    folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
    
    X_train = pd.read_csv(os.path.join(folder_name, 'X_train_f.csv'), index_col=0)
    y_train = pd.read_csv(os.path.join(folder_name, 'y_train_f.csv'), index_col=0)
    X_valid = pd.read_csv(os.path.join(folder_name, 'X_valid_f.csv'), index_col=0)
    y_valid = pd.read_csv(os.path.join(folder_name, 'y_valid_f.csv'), index_col=0)
    
    return X_train, y_train, X_valid, y_valid

base_folder = "AE_Data/Unfiltered_Shuffled_Compaction_Data"
X_train_loaded, y_train_loaded, X_valid_loaded, y_valid_loaded = load_datasets(fold=1, base_folder=base_folder)

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Define the output directory and subdirectories
output_dir = '8_Layer_AE_Default_Data/AE_Training_1'
subfolders = {
    'AE_Latent_Space_Data_TXT': os.path.join(output_dir, 'AE_Latent_Space_Data_TXT'),
    'AE_Training_Loss_Data': os.path.join(output_dir, 'AE_Training_Loss_Data'),
    'AE_Training_Loss_Graphs': os.path.join(output_dir, 'AE_Training_Loss_Graphs'),
    'AE_Latent_Space_Graphs': os.path.join(output_dir, 'AE_Latent_Space_Graphs'),
    'AE_Latent_Space_Data_PKL': os.path.join(output_dir, 'AE_Latent_Space_Data_PKL'),
    'models': os.path.join(output_dir, 'models'),
}

# Create all subdirectories
for folder in subfolders.values():
    os.makedirs(folder, exist_ok=True)

# Initialize the autoencoder with the loaded training data
autoencoder = AE_Model(X_train_loaded)

# Training loop
for counts in tqdm(range(20)):
    name = "Autoencoder_Training"

    # Save the latent space predictions log
    txt_file_path = os.path.join(subfolders['AE_Latent_Space_Data_TXT'], f"{counts}_Latent_Space_Predictions_Log.txt")
    with open(txt_file_path, "w") as file:
        # Train the model
        history = autoencoder.fit(
            X_train_loaded, X_train_loaded,
            epochs=1000,
            validation_data=(X_valid_loaded, X_valid_loaded),
            verbose=0
        )

        # Save training loss history
        training_history = pd.DataFrame(history.history)
        history_file_path = os.path.join(subfolders['AE_Training_Loss_Data'], f"{counts}_{name}_History.pkl")
        training_history.to_pickle(history_file_path)

        # Save training loss graph
        plt.plot(training_history)
        graph_file_path = os.path.join(subfolders['AE_Training_Loss_Graphs'], f"{counts}_{name}_History.png")
        plt.savefig(graph_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Define the latent space model
        dr_model = tf.keras.models.Model(
            inputs=autoencoder.input,
            outputs=autoencoder.get_layer('ae_latent').output
        )
        dr_model.summary(print_fn=lambda x: file.write(x + '\n'))

        # Initialize latent space variables
        x, y, z, indices = [], [], [], []

        # Process validation data in batches
        batch_size = 32
        for batch_start in range(0, len(X_valid_loaded), batch_size):
            batch_end = min(batch_start + batch_size, len(X_valid_loaded))
            X_batch = np.array(X_valid_loaded.iloc[batch_start:batch_end])
            y_batch = y_valid_loaded.iloc[batch_start:batch_end].values.flatten()

            # Predict latent space values
            op_batch = dr_model.predict(X_batch, verbose=0)
            for i, op in enumerate(op_batch):
                z.append(y_batch[i])
                x.append(op[0])
                y.append(op[1])
                indices.append(y_valid_loaded.index[batch_start + i])
                file.write(f"Prediction {batch_start + i}: {op}\n")

        # Save latent space scatter plot
        df = pd.DataFrame({'x': x, 'y': y, 'z': [f"trajectory-{k}" for k in z], 'index': indices})
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x='x', y='y', hue='z', data=df, s=10)
        scatter_file_path = os.path.join(subfolders['AE_Latent_Space_Graphs'], f"{counts}_Latent_Space.png")
        plt.savefig(scatter_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Save latent space data as pickle
        pkl_file_path = os.path.join(subfolders['AE_Latent_Space_Data_PKL'], f"{counts}_Latent_Space.pkl")
        df.to_pickle(pkl_file_path)

        # Save the model
        model_file_path = os.path.join(subfolders['models'], f"saved_model_{counts}.keras")
        save_model(autoencoder, model_file_path)

# 7 Layer AE

# Model 1

In [ ]:
# Save the model using the recommended Keras format
def save_model(model, model_path):
    if not model_path.endswith('.keras'):
        model_path += '.keras'
    model.save(model_path)
    
# Load the model
def load_existing_model(model_path):
    return load_model(model_path)

In [ ]:
# Get autoencoder model
def AE_Model(train_data, LeReLU_alpha=0.01, batch_size=256):
    
    input_layer = Input(shape=(train_data.shape[1],), name='ae_input')

    encoder = Dense(2048, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e1')(input_layer)
    encoder = Dense(1024, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e2')(encoder)
    encoder = Dense(512, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e3')(encoder)
    encoder = Dense(256, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e4')(encoder)
    encoder = Dense(128, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e5')(encoder)
    encoder = Dense(64, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e6')(encoder)
    encoder = Dense(32, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e7')(encoder)

    encoded = Dense(2, activation='linear', name='ae_latent')(encoder)
    
    decoder = Dense(32, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d7')(encoded)
    decoder = Dense(64, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d6')(decoder)
    decoder = Dense(128, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d5')(decoder)
    decoder = Dense(256, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d4')(decoder)
    decoder = Dense(512, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d3')(decoder)
    decoder = Dense(1024, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d2')(decoder)
    decoder = Dense(2048, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d1')(decoder)
    

    output_layer = Dense(train_data.shape[1], activation='linear', name='ae_output')(decoder)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=1e-5), loss=MeanSquaredError())
    
    model.summary()
    
    return model

In [ ]:
def load_datasets(fold, base_folder):
    """
    Load datasets for a specific fold.
    """
    folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
    
    X_train = pd.read_csv(os.path.join(folder_name, 'X_train_f.csv'), index_col=0)
    y_train = pd.read_csv(os.path.join(folder_name, 'y_train_f.csv'), index_col=0)
    X_valid = pd.read_csv(os.path.join(folder_name, 'X_valid_f.csv'), index_col=0)
    y_valid = pd.read_csv(os.path.join(folder_name, 'y_valid_f.csv'), index_col=0)
    
    return X_train, y_train, X_valid, y_valid

base_folder = "AE_Data/Filtered_Shuffled_Compaction_Data"
X_train_loaded, y_train_loaded, X_valid_loaded, y_valid_loaded = load_datasets(fold=1, base_folder=base_folder)

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Define the output directory and subdirectories
output_dir = '7_Layer_AE_Default_Data/AE_Training_1'
subfolders = {
    'AE_Latent_Space_Data_TXT': os.path.join(output_dir, 'AE_Latent_Space_Data_TXT'),
    'AE_Training_Loss_Data': os.path.join(output_dir, 'AE_Training_Loss_Data'),
    'AE_Training_Loss_Graphs': os.path.join(output_dir, 'AE_Training_Loss_Graphs'),
    'AE_Latent_Space_Graphs': os.path.join(output_dir, 'AE_Latent_Space_Graphs'),
    'AE_Latent_Space_Data_PKL': os.path.join(output_dir, 'AE_Latent_Space_Data_PKL'),
    'models': os.path.join(output_dir, 'models'),
}

# Create all subdirectories
for folder in subfolders.values():
    os.makedirs(folder, exist_ok=True)

# Initialize the autoencoder with the loaded training data
autoencoder = AE_Model(X_train_loaded)

# Training loop
for counts in tqdm(range(101)):
    name = "Autoencoder_Training"

    # Save the latent space predictions log
    txt_file_path = os.path.join(subfolders['AE_Latent_Space_Data_TXT'], f"{counts}_Latent_Space_Predictions_Log.txt")
    with open(txt_file_path, "w") as file:
        # Train the model
        history = autoencoder.fit(
            X_train_loaded, X_train_loaded,
            epochs=1000,
            validation_data=(X_valid_loaded, X_valid_loaded),
            verbose=0
        )

        # Save training loss history
        training_history = pd.DataFrame(history.history)
        history_file_path = os.path.join(subfolders['AE_Training_Loss_Data'], f"{counts}_{name}_History.pkl")
        training_history.to_pickle(history_file_path)

        # Save training loss graph
        plt.plot(training_history)
        graph_file_path = os.path.join(subfolders['AE_Training_Loss_Graphs'], f"{counts}_{name}_History.png")
        plt.savefig(graph_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Define the latent space model
        dr_model = tf.keras.models.Model(
            inputs=autoencoder.input,
            outputs=autoencoder.get_layer('ae_latent').output
        )
        dr_model.summary(print_fn=lambda x: file.write(x + '\n'))

        # Initialize latent space variables
        x, y, z, indices = [], [], [], []

        # Process validation data in batches
        batch_size = 32
        for batch_start in range(0, len(X_valid_loaded), batch_size):
            batch_end = min(batch_start + batch_size, len(X_valid_loaded))
            X_batch = np.array(X_valid_loaded.iloc[batch_start:batch_end])
            y_batch = y_valid_loaded.iloc[batch_start:batch_end].values.flatten()

            # Predict latent space values
            op_batch = dr_model.predict(X_batch, verbose=0)
            for i, op in enumerate(op_batch):
                z.append(y_batch[i])
                x.append(op[0])
                y.append(op[1])
                indices.append(y_valid_loaded.index[batch_start + i])
                file.write(f"Prediction {batch_start + i}: {op}\n")

        # Save latent space scatter plot
        df = pd.DataFrame({'x': x, 'y': y, 'z': [f"trajectory-{k}" for k in z], 'index': indices})
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x='x', y='y', hue='z', data=df, s=10)
        scatter_file_path = os.path.join(subfolders['AE_Latent_Space_Graphs'], f"{counts}_Latent_Space.png")
        plt.savefig(scatter_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Save latent space data as pickle
        pkl_file_path = os.path.join(subfolders['AE_Latent_Space_Data_PKL'], f"{counts}_Latent_Space.pkl")
        df.to_pickle(pkl_file_path)

        # Save the model
        model_file_path = os.path.join(subfolders['models'], f"saved_model_{counts}.keras")
        save_model(autoencoder, model_file_path)

# Reduce LR on Plateau

In [15]:
# Save the model using the recommended Keras format
def save_model(model, model_path):
    if not model_path.endswith('.keras'):
        model_path += '.keras'
    model.save(model_path)
    
# Load the model
def load_existing_model(model_path):
    return load_model(model_path)

In [16]:
# Get autoencoder model
def AE_Model(train_data, LeReLU_alpha=0.01, batch_size=256, learning_rate=1e-4):
    input_layer = Input(shape=(train_data.shape[1],), name='ae_input')
    
    encoder = Dense(1024, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e1')(input_layer)
    encoder = Dense(512, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e2')(encoder)
    encoder = Dense(256, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e3')(encoder)
    encoder = Dense(128, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e4')(encoder)
    encoder = Dense(64, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e5')(encoder)
    encoder = Dense(32, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e6')(encoder)

    encoded = Dense(2, activation='linear', name='ae_latent')(encoder)
    
    decoder = Dense(32, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d6')(encoded)
    decoder = Dense(64, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d5')(decoder)
    decoder = Dense(128, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d4')(decoder)
    decoder = Dense(256, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d3')(decoder)
    decoder = Dense(512, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d2')(decoder)
    decoder = Dense(1024, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d1')(decoder)

    output_layer = Dense(train_data.shape[1], activation='linear', name='ae_output')(decoder)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss=MeanSquaredError())
    
    model.summary()
    
    return model

# Model 1

In [17]:
base_folder = "AE_Data/Unfiltered_Shuffled_Compaction_Data"
X_train_loaded, y_train_loaded, X_valid_loaded, y_valid_loaded = load_datasets(fold=1, base_folder=base_folder)

In [18]:
## Learning Rate Multiply by 0.5 if sum val_loss (E900-E1000) / sum val_loss(E0-E100) > 0.9
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Define the output directory and subdirectories
output_dir = 'Default_AE_Default_Data_RLROP/AE_Training_1'
subfolders = {
    'AE_Latent_Space_Data_TXT': os.path.join(output_dir, 'AE_Latent_Space_Data_TXT'),
    'AE_Training_Loss_Data': os.path.join(output_dir, 'AE_Training_Loss_Data'),
    'AE_Training_Loss_Graphs': os.path.join(output_dir, 'AE_Training_Loss_Graphs'),
    'AE_Latent_Space_Graphs': os.path.join(output_dir, 'AE_Latent_Space_Graphs'),
    'AE_Latent_Space_Data_PKL': os.path.join(output_dir, 'AE_Latent_Space_Data_PKL'),
    'models': os.path.join(output_dir, 'models'),
}

# Create all subdirectories
for folder in subfolders.values():
    os.makedirs(folder, exist_ok=True)

# Initialize the autoencoder with the loaded training data
initial_learning_rate = 1e-4
min_learning_rate = 1e-6
autoencoder = AE_Model(X_train_loaded, learning_rate=initial_learning_rate)

# Initialize variables for RLROP
current_lr = initial_learning_rate

# Training loop
for counts in tqdm(range(101)):
    name = "Autoencoder_Training"

    # Save the latent space predictions log
    txt_file_path = os.path.join(subfolders['AE_Latent_Space_Data_TXT'], f"{counts}_Latent_Space_Predictions_Log.txt")
    with open(txt_file_path, "w") as file:
        # Train the model
        history = autoencoder.fit(
            X_train_loaded, X_train_loaded,
            epochs=1000,
            validation_data=(X_valid_loaded, X_valid_loaded),
            verbose=0
        )

        # Save training loss history
        training_history = pd.DataFrame(history.history)
        history_file_path = os.path.join(subfolders['AE_Training_Loss_Data'], f"{counts}_{name}_History.pkl")
        training_history.to_pickle(history_file_path)

        # Save training loss graph
        plt.plot(training_history['loss'], label='Train Loss')
        plt.plot(training_history['val_loss'], label='Validation Loss')
        plt.title(f"{name} (LR={current_lr:.1e})")
        plt.legend()
        graph_file_path = os.path.join(subfolders['AE_Training_Loss_Graphs'], f"{counts}_{name}_History.png")
        plt.savefig(graph_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Define the latent space model
        dr_model = tf.keras.models.Model(
            inputs=autoencoder.input,
            outputs=autoencoder.get_layer('ae_latent').output
        )
        dr_model.summary(print_fn=lambda x: file.write(x + '\n'))

        # Initialize latent space variables
        x, y, z, indices = [], [], [], []

        # Process validation data in batches
        batch_size = 32
        for batch_start in range(0, len(X_valid_loaded), batch_size):
            batch_end = min(batch_start + batch_size, len(X_valid_loaded))
            X_batch = np.array(X_valid_loaded.iloc[batch_start:batch_end])
            y_batch = y_valid_loaded.iloc[batch_start:batch_end].values.flatten()

            # Predict latent space values
            op_batch = dr_model.predict(X_batch, verbose=0)
            for i, op in enumerate(op_batch):
                z.append(y_batch[i])
                x.append(op[0])
                y.append(op[1])
                indices.append(y_valid_loaded.index[batch_start + i])
                file.write(f"Prediction {batch_start + i}: {op}\n")

        # Save latent space scatter plot
        df = pd.DataFrame({'x': x, 'y': y, 'z': [f"trajectory-{k}" for k in z], 'index': indices})
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x='x', y='y', hue='z', data=df, s=10)
        scatter_file_path = os.path.join(subfolders['AE_Latent_Space_Graphs'], f"{counts}_Latent_Space.png")
        plt.savefig(scatter_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Save latent space data as pickle
        pkl_file_path = os.path.join(subfolders['AE_Latent_Space_Data_PKL'], f"{counts}_Latent_Space.pkl")
        df.to_pickle(pkl_file_path)

        # Save the model
        model_file_path = os.path.join(subfolders['models'], f"saved_model_{counts}.keras")
        save_model(autoencoder, model_file_path)

        # Implement RLROP: Calculate average validation loss reduction
        if counts > 0:
            first_100_avg = training_history['val_loss'][:100].mean()
            last_100_avg = training_history['val_loss'][-100:].mean()
            if (first_100_avg - last_100_avg) / first_100_avg < 0.1:
                current_lr = max(current_lr * 0.5, min_learning_rate)
                autoencoder.optimizer.learning_rate.assign(current_lr)

Model: "functional_108"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ ae_input (InputLayer)           │ (None, 2175)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e1 (Dense)                      │ (None, 1024)           │     2,228,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e2 (Dense)                      │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e3 (Dense)                      │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e4 (Dense)                      │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e5 (Dense)                      │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e6 (Dense)                      │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ae_latent (Dense)               │ (None, 2)              │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d6 (Dense)                      │ (None, 32)             │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d5 (Dense)                      │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d4 (Dense)                      │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d3 (Dense)                      │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d2 (Dense)                      │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d1 (Dense)                      │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ae_output (Dense)               │ (None, 2175)           │     2,229,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,857,473 (22.34 MB)

 Trainable params: 5,857,473 (22.34 MB)

 Non-trainable params: 0 (0.00 B)

...

 96%|████████████████████████████████████▍ | 97/101 [21:00:56<52:08, 782.08s/it]

 97%|████████████████████████████████████▊ | 98/101 [21:14:02<39:09, 783.11s/it]

 98%|█████████████████████████████████████▏| 99/101 [21:27:01<26:03, 781.86s/it]

 99%|████████████████████████████████████▋| 100/101 [21:39:58<13:00, 780.57s/it]

100%|█████████████████████████████████████| 101/101 [21:52:58<00:00, 779.99s/it]


# Shuffled, Filtered Dataset

In [2]:
# Save the model using the recommended Keras format
def save_model(model, model_path):
    if not model_path.endswith('.keras'):
        model_path += '.keras'
    model.save(model_path)
    
# Load the model
def load_existing_model(model_path):
    return load_model(model_path)

In [3]:
# Get autoencoder model
def AE_Model(train_data, LeReLU_alpha=0.01, batch_size=256):
    
    input_layer = Input(shape=(train_data.shape[1],), name='ae_input')
    
    encoder = Dense(1024, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e1')(input_layer)
    encoder = Dense(512, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e2')(encoder)
    encoder = Dense(256, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e3')(encoder)
    encoder = Dense(128, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e4')(encoder)
    encoder = Dense(64, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e5')(encoder)
    encoder = Dense(32, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='e6')(encoder)

    encoded = Dense(2, activation='linear', name='ae_latent')(encoder)
    
    decoder = Dense(32, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d6')(encoded)
    decoder = Dense(64, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d5')(decoder)
    decoder = Dense(128, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d4')(decoder)
    decoder = Dense(256, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d3')(decoder)
    decoder = Dense(512, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d2')(decoder)
    decoder = Dense(1024, activation=LeakyReLU(negative_slope=LeReLU_alpha), name='d1')(decoder)

    output_layer = Dense(train_data.shape[1], activation='linear', name='ae_output')(decoder)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=1e-5), loss=MeanSquaredError())
    
    model.summary()
    
    return model

In [4]:
def load_datasets(fold, base_folder):
    """
    Load datasets for a specific fold.
    """
    folder_name = os.path.join(base_folder, f'Training_Set_{fold}')
    
    X_train = pd.read_csv(os.path.join(folder_name, 'X_train_f.csv'), index_col=0)
    y_train = pd.read_csv(os.path.join(folder_name, 'y_train_f.csv'), index_col=0)
    X_valid = pd.read_csv(os.path.join(folder_name, 'X_valid_f.csv'), index_col=0)
    y_valid = pd.read_csv(os.path.join(folder_name, 'y_valid_f.csv'), index_col=0)
    
    return X_train, y_train, X_valid, y_valid

base_folder = "AE_Data/Filtered_Shuffled_Compaction_Data"
X_train_loaded, y_train_loaded, X_valid_loaded, y_valid_loaded = load_datasets(fold=1, base_folder=base_folder)

In [5]:
print(X_train_loaded)

             10        26        29        30        38         7         8  \
32823  0.581820  0.580513  0.465041  0.219771  0.394883  0.369550  0.414636   
28505  0.605215  0.357156  0.805869  0.619109  0.487875  0.147931  0.142953   
6689   0.458832  0.222296  0.751903  0.273659  0.483952  0.260578  0.636252   
36572  0.242426  0.269379  0.401734  0.490664  0.503974  0.245061  0.803909   
12335  0.546329  0.397127  0.774965  0.125088  0.447993  0.149885  0.678769   
...         ...       ...       ...       ...       ...       ...       ...   
6265   0.625532  0.281942  0.681292  0.164125  0.666801  0.155468  0.574162   
11284  0.532060  0.722206  0.678063  0.189079  0.550389  0.267015  0.705913   
38158  0.247786  0.795716  0.586447  0.505211  0.549095  0.369128  0.542627   
860    0.467415  0.540923  0.864833  0.199565  0.800465  0.178709  0.074222   
15795  0.530086  0.638857  0.831911  0.242039  0.624472  0.318827  0.628040   

             28        43       8.1  ...         2 

In [6]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Define the output directory and subdirectories
output_dir = 'Default_AE_Filtered_Data/AE_Training_1'
subfolders = {
    'AE_Latent_Space_Data_TXT': os.path.join(output_dir, 'AE_Latent_Space_Data_TXT'),
    'AE_Training_Loss_Data': os.path.join(output_dir, 'AE_Training_Loss_Data'),
    'AE_Training_Loss_Graphs': os.path.join(output_dir, 'AE_Training_Loss_Graphs'),
    'AE_Latent_Space_Graphs': os.path.join(output_dir, 'AE_Latent_Space_Graphs'),
    'AE_Latent_Space_Data_PKL': os.path.join(output_dir, 'AE_Latent_Space_Data_PKL'),
    'models': os.path.join(output_dir, 'models'),
}

# Create all subdirectories
for folder in subfolders.values():
    os.makedirs(folder, exist_ok=True)

# Initialize the autoencoder with the loaded training data
autoencoder = AE_Model(X_train_loaded)

# Training loop
for counts in tqdm(range(101)):
    name = "Autoencoder_Training"

    # Save the latent space predictions log
    txt_file_path = os.path.join(subfolders['AE_Latent_Space_Data_TXT'], f"{counts}_Latent_Space_Predictions_Log.txt")
    with open(txt_file_path, "w") as file:
        # Train the model
        history = autoencoder.fit(
            X_train_loaded, X_train_loaded,
            epochs=1000,
            validation_data=(X_valid_loaded, X_valid_loaded),
            verbose=0
        )

        # Save training loss history
        training_history = pd.DataFrame(history.history)
        history_file_path = os.path.join(subfolders['AE_Training_Loss_Data'], f"{counts}_{name}_History.pkl")
        training_history.to_pickle(history_file_path)

        # Save training loss graph
        plt.plot(training_history)
        graph_file_path = os.path.join(subfolders['AE_Training_Loss_Graphs'], f"{counts}_{name}_History.png")
        plt.savefig(graph_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Define the latent space model
        dr_model = tf.keras.models.Model(
            inputs=autoencoder.input,
            outputs=autoencoder.get_layer('ae_latent').output
        )
        dr_model.summary(print_fn=lambda x: file.write(x + '\n'))

        # Initialize latent space variables
        x, y, z, indices = [], [], [], []

        # Process validation data in batches
        batch_size = 32
        for batch_start in range(0, len(X_valid_loaded), batch_size):
            batch_end = min(batch_start + batch_size, len(X_valid_loaded))
            X_batch = np.array(X_valid_loaded.iloc[batch_start:batch_end])
            y_batch = y_valid_loaded.iloc[batch_start:batch_end].values.flatten()

            # Predict latent space values
            op_batch = dr_model.predict(X_batch, verbose=0)
            for i, op in enumerate(op_batch):
                z.append(y_batch[i])
                x.append(op[0])
                y.append(op[1])
                indices.append(y_valid_loaded.index[batch_start + i])
                file.write(f"Prediction {batch_start + i}: {op}\n")

        # Save latent space scatter plot
        df = pd.DataFrame({'x': x, 'y': y, 'z': [f"trajectory-{k}" for k in z], 'index': indices})
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x='x', y='y', hue='z', data=df, s=10)
        scatter_file_path = os.path.join(subfolders['AE_Latent_Space_Graphs'], f"{counts}_Latent_Space.png")
        plt.savefig(scatter_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Save latent space data as pickle
        pkl_file_path = os.path.join(subfolders['AE_Latent_Space_Data_PKL'], f"{counts}_Latent_Space.pkl")
        df.to_pickle(pkl_file_path)

        # Save the model
        model_file_path = os.path.join(subfolders['models'], f"saved_model_{counts}.keras")
        save_model(autoencoder, model_file_path)

I0000 00:00:1733076887.172368    4131 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733076887.239965    4131 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733076887.240081    4131 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733076887.241761    4131 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ ae_input (InputLayer)           │ (None, 122)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e1 (Dense)                      │ (None, 1024)           │       125,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e2 (Dense)                      │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e3 (Dense)                      │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e4 (Dense)                      │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e5 (Dense)                      │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e6 (Dense)                      │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ae_latent (Dense)               │ (None, 2)              │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d6 (Dense)                      │ (None, 32)             │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d5 (Dense)                      │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d4 (Dense)                      │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d3 (Dense)                      │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d2 (Dense)                      │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ d1 (Dense)                      │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ae_output (Dense)               │ (None, 122)            │       125,050 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,650,876 (6.30 MB)

 Trainable params: 1,650,876 (6.30 MB)

...

 96%|████████████████████████████████████▍ | 97/101 [14:15:35<35:35, 533.89s/it]

 97%|████████████████████████████████████▊ | 98/101 [14:24:32<26:43, 534.59s/it]

 98%|█████████████████████████████████████▏| 99/101 [14:33:20<17:45, 532.80s/it]

 99%|████████████████████████████████████▋| 100/101 [14:41:56<08:47, 527.67s/it]

100%|█████████████████████████████████████| 101/101 [14:50:32<00:00, 529.03s/it]


# Restart Training

In [ ]:
# base_folder = "AE_Data/Filtered_Shuffled_Compaction_Data"
X_train_loaded, y_train_loaded, X_valid_loaded, y_valid_loaded = load_datasets(fold=1, base_folder=base_folder)

In [ ]:
def save_model(model, model_path):
    if not model_path.endswith('.keras'):
        model_path += '.keras'
    model.save(model_path)

def load_existing_model(model_path):
    return load_model(model_path)

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import load_model
import numpy as np

# Define the output directory and subdirectories
# output_dir = 'Default_AE_Default_Data/AE_Training_X'
subfolders = {
    'AE_Latent_Space_Data_TXT': os.path.join(output_dir, 'AE_Latent_Space_Data_TXT'),
    'AE_Training_Loss_Data': os.path.join(output_dir, 'AE_Training_Loss_Data'),
    'AE_Training_Loss_Graphs': os.path.join(output_dir, 'AE_Training_Loss_Graphs'),
    'AE_Latent_Space_Graphs': os.path.join(output_dir, 'AE_Latent_Space_Graphs'),
    'AE_Latent_Space_Data_PKL': os.path.join(output_dir, 'AE_Latent_Space_Data_PKL'),
    'models': os.path.join(output_dir, 'models'),
}

# Create all subdirectories if they don't exist
for folder in subfolders.values():
    os.makedirs(folder, exist_ok=True)

# Load the model from the last saved checkpoint
# checkpoint_epoch_set = XXX
model_file_path = os.path.join(subfolders['models'], f"saved_model_{checkpoint_epoch_set}.keras")
autoencoder = load_model(model_file_path)

# Resume training loop starting from the specified epoch set
for counts in tqdm(range(checkpoint_epoch_set + 1, 101)):
    name = "Autoencoder_Training"

    # Save the latent space predictions log
    txt_file_path = os.path.join(subfolders['AE_Latent_Space_Data_TXT'], f"{counts}_Latent_Space_Predictions_Log.txt")
    with open(txt_file_path, "w") as file:
        # Train the model
        history = autoencoder.fit(
            X_train_loaded, X_train_loaded,
            epochs=1000,
            validation_data=(X_valid_loaded, X_valid_loaded),
            verbose=0
        )

        # Save training loss history
        training_history = pd.DataFrame(history.history)
        history_file_path = os.path.join(subfolders['AE_Training_Loss_Data'], f"{counts}_{name}_History.pkl")
        training_history.to_pickle(history_file_path)

        # Save training loss graph
        plt.plot(training_history)
        graph_file_path = os.path.join(subfolders['AE_Training_Loss_Graphs'], f"{counts}_{name}_History.png")
        plt.savefig(graph_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Define the latent space model
        dr_model = tf.keras.models.Model(
            inputs=autoencoder.input,
            outputs=autoencoder.get_layer('ae_latent').output
        )
        dr_model.summary(print_fn=lambda x: file.write(x + '\n'))

        # Initialize latent space variables
        x, y, z, indices = [], [], [], []

        # Process validation data in batches
        batch_size = 32
        for batch_start in range(0, len(X_valid_loaded), batch_size):
            batch_end = min(batch_start + batch_size, len(X_valid_loaded))
            X_batch = np.array(X_valid_loaded.iloc[batch_start:batch_end])
            y_batch = y_valid_loaded.iloc[batch_start:batch_end].values.flatten()

            # Predict latent space values
            op_batch = dr_model.predict(X_batch, verbose=0)
            for i, op in enumerate(op_batch):
                z.append(y_batch[i])
                x.append(op[0])
                y.append(op[1])
                indices.append(y_valid_loaded.index[batch_start + i])
                file.write(f"Prediction {batch_start + i}: {op}\n")

        # Save latent space scatter plot
        df = pd.DataFrame({'x': x, 'y': y, 'z': [f"trajectory-{k}" for k in z], 'index': indices})
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x='x', y='y', hue='z', data=df, s=10)
        scatter_file_path = os.path.join(subfolders['AE_Latent_Space_Graphs'], f"{counts}_Latent_Space.png")
        plt.savefig(scatter_file_path, dpi=300)
        plt.clf()
        plt.close()

        # Save latent space data as pickle
        pkl_file_path = os.path.join(subfolders['AE_Latent_Space_Data_PKL'], f"{counts}_Latent_Space.pkl")
        df.to_pickle(pkl_file_path)

        # Save the model
        model_file_path = os.path.join(subfolders['models'], f"saved_model_{counts}.keras")
        autoencoder.save(model_file_path)

# Original Training Code

# Generating Full Latent Space Representations:

In [ ]:
# Concatenate training and validation datasets
X_loaded = pd.concat([X_train_loaded, X_valid_loaded], axis=0)

In [ ]:
# Function to create the latent space representations and save to CSV
def generate_and_save_latent_space_representations(model_path, data, output_dir, output_filename):
    # Load the saved model
    autoencoder = load_model(model_path)

    # Create a model to get the latent space representation
    dr_model = tf.keras.models.Model(inputs=autoencoder.get_layer('ae_input').input,
                                     outputs=autoencoder.get_layer('ae_latent').output)
    
    # Generate latent space representations for the entire dataset
    latent_space_representations = dr_model.predict(data, verbose=1)

    # Create a DataFrame to store the representations with the index from the input data
    df_latent_space = pd.DataFrame(latent_space_representations, columns=['x', 'y'], index=data.index)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Save the DataFrame to a CSV file
    output_filepath = os.path.join(output_dir, output_filename)
    df_latent_space.to_pickle(output_filepath)

    print(f"Latent space representations saved to {output_filepath}")

# Specify the path to the saved model
model_path = 'AE_Training_1/models/saved_model_20.keras'

# Specify the output directory and filename
output_dir = 'Full_LSPs_Same_Training_Data'
output_filename = 'AE1_ES_20_LSR_Full'

# Generate and save the latent space representations
generate_and_save_latent_space_representations(model_path, X_loaded, output_dir, output_filename)

In [ ]:
# Function to create the latent space representations and save to CSV
def generate_and_save_latent_space_representations(model_path, data, output_dir, output_filename):
    # Load the saved model
    autoencoder = load_model(model_path)

    # Create a model to get the latent space representation
    dr_model = tf.keras.models.Model(inputs=autoencoder.get_layer('ae_input').input,
                                     outputs=autoencoder.get_layer('ae_latent').output)
    
    # Generate latent space representations for the entire dataset
    latent_space_representations = dr_model.predict(data, verbose=1)

    # Create a DataFrame to store the representations with the index from the input data
    df_latent_space = pd.DataFrame(latent_space_representations, columns=['x', 'y'], index=data.index)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Save the DataFrame to a CSV file
    output_filepath = os.path.join(output_dir, output_filename)
    df_latent_space.to_pickle(output_filepath)

    print(f"Latent space representations saved to {output_filepath}")

# Specify the path to the saved model
model_path = 'AE_Training_2/models/saved_model_20.keras'

# Specify the output directory and filename
output_dir = 'Full_LSPs_Same_Training_Data'
output_filename = 'AE2_ES_20_LSR_Full'

# Generate and save the latent space representations
generate_and_save_latent_space_representations(model_path, X_loaded, output_dir, output_filename)

In [ ]:
# Function to create the latent space representations and save to CSV
def generate_and_save_latent_space_representations(model_path, data, output_dir, output_filename):
    # Load the saved model
    autoencoder = load_model(model_path)

    # Create a model to get the latent space representation
    dr_model = tf.keras.models.Model(inputs=autoencoder.get_layer('ae_input').input,
                                     outputs=autoencoder.get_layer('ae_latent').output)
    
    # Generate latent space representations for the entire dataset
    latent_space_representations = dr_model.predict(data, verbose=1)

    # Create a DataFrame to store the representations with the index from the input data
    df_latent_space = pd.DataFrame(latent_space_representations, columns=['x', 'y'], index=data.index)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Save the DataFrame to a CSV file
    output_filepath = os.path.join(output_dir, output_filename)
    df_latent_space.to_pickle(output_filepath)

    print(f"Latent space representations saved to {output_filepath}")

# Specify the path to the saved model
model_path = 'AE_Training_3/models/saved_model_20.keras'

# Specify the output directory and filename
output_dir = 'Full_LSPs_Same_Training_Data'
output_filename = 'AE3_ES_20_LSR_Full'

# Generate and save the latent space representations
generate_and_save_latent_space_representations(model_path, X_loaded, output_dir, output_filename)

In [ ]:
# Function to create the latent space representations and save to CSV
def generate_and_save_latent_space_representations(model_path, data, output_dir, output_filename):
    # Load the saved model
    autoencoder = load_model(model_path)

    # Create a model to get the latent space representation
    dr_model = tf.keras.models.Model(inputs=autoencoder.get_layer('ae_input').input,
                                     outputs=autoencoder.get_layer('ae_latent').output)
    
    # Generate latent space representations for the entire dataset
    latent_space_representations = dr_model.predict(data, verbose=1)

    # Create a DataFrame to store the representations with the index from the input data
    df_latent_space = pd.DataFrame(latent_space_representations, columns=['x', 'y'], index=data.index)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Save the DataFrame to a CSV file
    output_filepath = os.path.join(output_dir, output_filename)
    df_latent_space.to_pickle(output_filepath)

    print(f"Latent space representations saved to {output_filepath}")

# Specify the path to the saved model
model_path = 'AE_Training_4/models/saved_model_20.keras'

# Specify the output directory and filename
output_dir = 'Full_LSPs_Same_Training_Data'
output_filename = 'AE4_ES_20_LSR_Full'

# Generate and save the latent space representations
generate_and_save_latent_space_representations(model_path, X_loaded, output_dir, output_filename)

# Load Latent Space Data For Clustering

In [ ]:
import os
import gc
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from matplotlib.image import imread
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, fcluster

import MDAnalysis as mda
from itertools import combinations
from mpl_toolkits.mplot3d import Axes3D
from MDAnalysis.analysis import align, rms, distances

# Load the topology and trajectory files for both MD simulations
u1 = mda.Universe("myc_091-160_stripped.prmtop", "aMD_091-160_no1-4_stripped.nc")
u2 = mda.Universe("myc_091-160_D132-H_stripped.prmtop", "aMD_091-160_D132-H_no1-4_stripped.nc")

In [ ]:
# Latent Space 1
folder_name_1 = 'Full_LSPs'
file_name_1 = 'AE_Fold_1_ES_6.csv'
latent_1 = pd.read_csv(f'{folder_name_1}/{file_name_1}')

# Latent Space 2
folder_name_2 = 'Full_LSPs'
file_name_2 = 'AE_Fold_2_ES_6.csv'
latent_2 = pd.read_csv(f'{folder_name_2}/{file_name_2}')

# Latent Space 3
folder_name_3 = 'Full_LSPs'
file_name_3 = 'AE_Fold_3_ES_6.csv'
latent_3 = pd.read_csv(f'{folder_name_3}/{file_name_3}')

# Latent Space 4
folder_name_4 = 'Full_LSPs'
file_name_4 = 'AE_Fold_4_ES_6.csv'
latent_4 = pd.read_csv(f'{folder_name_4}/{file_name_4}')

# Latent Space 5
folder_name_5 = 'Full_LSPs'
file_name_5 = 'AE_Fold_5_ES_6.csv'
latent_5 = pd.read_csv(f'{folder_name_5}/{file_name_5}')

# Clustering (Batched)

In [ ]:
threshold_val = 1.0
length_val = 1

# Function to perform clustering in batches
def batch_clustering(data, batch_size, threshold):
    clusters = []
    start = 0
    while start < data.shape[0]:
        end = min(start + batch_size, data.shape[0])
        batch = data[start:end]
        linkage_matrix = linkage(batch, method='ward')
        batch_clusters = fcluster(linkage_matrix, t=threshold, criterion='distance')
        clusters.extend(batch_clusters)
        start = end
    return np.array(clusters)

# Define a function to process and cluster each latent space in batches
def process_and_cluster_in_batches(latent_df, scaler, batch_size, threshold=threshold_val):
    coords = latent_df[['x', 'y']].values
    coords_scaled = scaler.fit_transform(coords)
    clusters = batch_clustering(coords_scaled, batch_size, threshold)
    return clusters

# Initialize the scaler
scaler = StandardScaler()

# List of latent DataFrames
latent_dfs = [latent_1, latent_2, latent_3, latent_4, latent_5]

# Dictionary to store clusters
clusters_dict = {}

# Define batch size and threshold
batch_size = 20000
threshold = threshold_val

# Process and cluster each latent DataFrame in batches
for i, latent_df in enumerate(latent_dfs, start=1):
    clusters_dict[f'clusters{i}'] = process_and_cluster_in_batches(latent_df, scaler, batch_size, threshold)

# Combine the cluster assignments from all latent spaces
combined_clusters = np.vstack(list(clusters_dict.values())).T

# Find consensus clusters
consensus_clusters = {}
for i, cluster_tuple in enumerate(combined_clusters):
    cluster_tuple = tuple(cluster_tuple)
    if cluster_tuple not in consensus_clusters:
        consensus_clusters[cluster_tuple] = []
    consensus_clusters[cluster_tuple].append(int(latent_1.iloc[i, 0]))

# Filter out singleton clusters (clusters with only one member)
consensus_clusters = {k: v for k, v in consensus_clusters.items() if len(v) > length_val}

# Calculate cluster sizes for plotting
cluster_sizes = {k: len(v) for k, v in consensus_clusters.items()}

# Visualization function
def plot_clusters(latent, clusters, cluster_sizes, title):
    cluster_count = pd.Series(clusters).value_counts()
    sizes = cluster_count.loc[clusters].values
    
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(latent['x'], latent['y'], c=clusters, s=sizes, cmap='viridis', marker='o', alpha=0.6)
    plt.title(title)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.colorbar(scatter, label='Cluster')
    plt.show()

# Add cluster labels to the DataFrames for plotting and plot each
for i, latent_df in enumerate(latent_dfs, start=1):
    latent_df['cluster'] = clusters_dict[f'clusters{i}']
    plot_clusters(latent_df, latent_df['cluster'], cluster_sizes, f'Clusters in Latent Space {i}')

In [ ]:
# Group the DataFrame by 'cluster' and print the indices sorted by the cluster
clustered_indices = latent_1.groupby('cluster', group_keys=False).apply(lambda df: df.index.tolist(), include_groups=False)

# Print the indices sorted by the cluster
for cluster_id, indices in clustered_indices.items():
    print(f"Cluster {cluster_id}: {indices}")

In [ ]:
# Group the DataFrame by 'cluster' and count the number of indices in each cluster
cluster_counts = latent_1['cluster'].value_counts().sort_values()

# Print the clusters and their counts sorted by the number of indices in ascending order
for cluster_id, count in cluster_counts.items():
    print(f"Cluster {cluster_id}: {count}")

# Generate Cluster Statistics

In [ ]:
import os
import pandas as pd
import numpy as np
import MDAnalysis as mda
from MDAnalysis.analysis import align
from tqdm import tqdm

threshold_val = 1.0
length_val = 1
minimum_indices = 1

# Create the directory to save the files
output_dir = f"N4_Threshold_{threshold_val}_Length_{length_val}"
os.makedirs(output_dir, exist_ok=True)

# Function to extract frames for residues 15-30
def extract_frames(indices, universe1, universe2, switch_index=40000, selection="resid 15-30 and name CA"):
    frames = []
    for idx in indices:
        universe = universe1 if idx < switch_index else universe2
        frame_idx = idx if idx < switch_index else idx - switch_index
        universe.trajectory[frame_idx]
        frames.append(universe.select_atoms(selection).positions.copy())
    return frames

# Function to create a temporary universe
def create_temp_universe(positions, universe, selection="resid 15-30 and name CA"):
    temp_universe = mda.Merge(universe.select_atoms(selection))
    temp_universe.atoms.positions = positions
    return temp_universe

# Function to align frames and calculate normalized RMSD
def align_and_calculate_rmsd(reference_universe, frames, select_for_alignment):
    rmsd_list = []
    for frame_positions in frames:
        temp_universe = create_temp_universe(frame_positions, reference_universe, selection=select_for_alignment)
        aligner = align.AlignTraj(temp_universe, reference_universe, select=select_for_alignment, in_memory=True)
        aligner.run()
        rmsd = align.rms.rmsd(temp_universe.atoms.positions, reference_universe.atoms.positions)
        rmsd_normalized = rmsd / np.linalg.norm(reference_universe.atoms.positions)
        rmsd_list.append(np.mean(rmsd_normalized))
    return np.mean(rmsd_list)

# Define selection string for alignment
select_residues_15_30 = "resid 15-30 and name CA"

# Prepare the alignment CSV file
alignment_file = os.path.join(output_dir, "Latent_Alignment.csv")
alignment_cols = ['index', 'x', 'y', 'cluster', 'reference_frame']

# Initialize the reference frames dictionary
reference_frames = {}

# Check if the file exists and load existing data
if os.path.exists(alignment_file):
    existing_data = pd.read_csv(alignment_file)
    last_cluster_id = existing_data['cluster'].max()
else:
    with open(alignment_file, 'w') as f:
        f.write(','.join(alignment_cols) + '\n')
    existing_data = pd.DataFrame(columns=alignment_cols)
    last_cluster_id = -1

# Calculate reference frames and update the alignment CSV file incrementally
for cluster_id, cluster_data in tqdm(latent_1.groupby('cluster'), desc="Analyzing clusters", unit="cluster"):
    if cluster_id <= last_cluster_id:
        continue

    indices = cluster_data.index.tolist()
    if len(indices) < minimum_indices:
        continue 

    frames = extract_frames(indices, u1, u2, selection=select_residues_15_30)
    
    # Determine the reference frame that minimizes the RMSD
    min_rmsd = float('inf')
    reference_frame_idx = -1
    for i, ref_frame_positions in enumerate(frames):
        reference_universe = create_temp_universe(ref_frame_positions, u1, selection=select_residues_15_30)
        rmsd = align_and_calculate_rmsd(reference_universe, frames, select_residues_15_30)
        if rmsd < min_rmsd:
            min_rmsd = rmsd
            reference_frame_idx = indices[i]
    
    reference_frames[cluster_id] = reference_frame_idx
    
    # Update the alignment data
    cluster_data['reference_frame'] = reference_frame_idx
    cluster_data['index'] = cluster_data.index
    
    # Append the alignment data to the CSV file
    cluster_data[alignment_cols].to_csv(alignment_file, mode='a', header=False, index=False)

# Perform statistical analysis on the clusters
latent_1_sorted = latent_1.sort_index()
cluster_durations = []
current_cluster = None
current_start_index = None

# Iterate through the sorted DataFrame to find cluster durations
for idx, row in latent_1_sorted.iterrows():
    cluster = row['cluster']
    if cluster == current_cluster:
        current_end_index = idx
    else:
        if current_cluster is not None:
            duration = current_end_index - current_start_index + 1
            cluster_durations.append((current_cluster, duration))
        current_cluster = cluster
        current_start_index = idx
        current_end_index = idx

# Append the last cluster duration
if current_cluster is not None:
    duration = current_end_index - current_start_index + 1
    cluster_durations.append((current_cluster, duration))

# Convert durations to DataFrame
durations_df = pd.DataFrame(cluster_durations, columns=['Cluster', 'Duration'])

# Group by cluster and calculate summary metrics
summary_df = durations_df.groupby('Cluster').agg(
    Entries=('Cluster', 'size'),
    Tot_Dur=('Duration', 'sum'),
    Max_Dur=('Duration', 'max'),
    Av_Dur=('Duration', 'mean')
).reset_index().sort_values(by='Cluster')

# Save the summary DataFrame to CSV
statistics_file = os.path.join(output_dir, "Cluster_Statistics.csv")
summary_df.to_csv(statistics_file, index=False)

# Display the summary DataFrame
pd.set_option('display.max_rows', None)
print(summary_df)
pd.reset_option('display.max_rows')

In [ ]:
import os
import numpy as np
import pandas as pd
import MDAnalysis as mda
from MDAnalysis.analysis import align, rms, distances
import gc
from tqdm import tqdm

threshold_val = 1.0
length_val = 1
minimum_indices = 1

# Define the output directories
output_dir = f"N4_Threshold_{threshold_val}_Length_{length_val}"
expanded_csv_path = os.path.join(output_dir, "Cluster_Statistics_Expanded.csv")

# Load the CSV files
latent_1_alignment = pd.read_csv(os.path.join(output_dir, "Latent_Alignment.csv"))
latent_1_statistics = pd.read_csv(os.path.join(output_dir, "Cluster_Statistics.csv"))

# Load the MD simulation data
u1 = mda.Universe("myc_091-160_stripped.prmtop", "aMD_091-160_no1-4_stripped.nc")
u2 = mda.Universe("myc_091-160_D132-H_stripped.prmtop", "aMD_091-160_D132-H_no1-4_stripped.nc")

# Define the function to extract frames from both universes
def extract_frames(indices, universe1, universe2, switch_index=40000):
    frames = []
    for idx in indices:
        if idx < switch_index:
            universe1.trajectory[idx]
            frames.append(universe1.select_atoms("name CA").positions.copy())
        else:
            universe2.trajectory[idx - switch_index]
            frames.append(universe2.select_atoms("name CA").positions.copy())
    return frames

# Create a temporary Universe for alignment
def create_temp_universe(positions, universe):
    temp_universe = mda.Merge(universe.select_atoms("name CA"))
    temp_universe.atoms.positions = positions
    return temp_universe

# Align frames using AlignTraj
def align_frames(reference_universe, frames, select_for_alignment):
    aligned_frames = []
    for frame_positions in frames:
        temp_universe = create_temp_universe(frame_positions, reference_universe)
        aligner = align.AlignTraj(temp_universe, reference_universe, select=select_for_alignment, in_memory=False)
        aligner.run()
        aligned_frames.append(temp_universe.atoms.positions.copy())
        del temp_universe  # Explicitly delete to free memory
        gc.collect()
    return aligned_frames

# Calculate RMSD for aligned frames
def calculate_rmsd(reference_positions, frames):
    rmsd_values = [rms.rmsd(frame, reference_positions) for frame in frames]
    return np.mean(rmsd_values), np.std(rmsd_values)

# Measure distances between residues
def calculate_distances(universe, res1, res2):
    distance_values = distances.distance_array(
        universe.select_atoms(f"resid {res1} and name CA").positions,
        universe.select_atoms(f"resid {res2} and name CA").positions
    )
    mean_distance = np.mean(distance_values)
    return mean_distance

# Selection string for residues 15-30 to ensure alignment of corresponding residues
select_residues_15_30 = "resid 15-30 and name CA"

# Create a function to calculate cluster statistics
def calculate_cluster_statistics(cluster_id, indices, reference_idx):
    # Extract frames for the cluster
    frames = extract_frames(indices, u1, u2)
    
    # Extract the reference frame for alignment
    if reference_idx < 40000:
        u1.trajectory[reference_idx]
        reference_positions = u1.select_atoms("name CA").positions.copy()
    else:
        u2.trajectory[reference_idx - 40000]
        reference_positions = u2.select_atoms("name CA").positions.copy()
    
    reference_universe = create_temp_universe(reference_positions, u1)
    
    # Align frames using the specific residues 15-30
    aligned_frames = align_frames(reference_universe, frames, select_residues_15_30)
    
    # Calculate RMSD
    mean_rmsd, std_rmsd = calculate_rmsd(reference_positions, aligned_frames)
    
    # Measure distances
    mean_dist_15_22 = calculate_distances(reference_universe, 15, 22)
    mean_dist_22_30 = calculate_distances(reference_universe, 22, 30)
    mean_dist_15_30 = calculate_distances(reference_universe, 15, 30)
    
    # Save the statistics to a list for the new DataFrame
    cluster_stats = {
        "Cluster": cluster_id,
        "Frames": len(indices),
        "Mean_RMSD": mean_rmsd,
        "Std_RMSD": std_rmsd,
        "Mean_Dist_15_22": mean_dist_15_22,
        "Mean_Dist_22_30": mean_dist_22_30,
        "Mean_Dist_15_30": mean_dist_15_30,
    }
    
    return cluster_stats

# Function to load existing cluster statistics if the CSV file exists
def load_existing_stats(file_path):
    if os.path.exists(file_path):
        return pd.read_csv(file_path)
    else:
        return pd.DataFrame()

# Load existing statistics
existing_stats_df = load_existing_stats(expanded_csv_path)
processed_clusters = set(existing_stats_df['Cluster']) if not existing_stats_df.empty else set()

# Group the DataFrame by 'cluster' and get the indices for each cluster
cluster_indices = latent_1_alignment.groupby('cluster').apply(lambda df: df['index'].tolist()).to_dict()

# Get the best reference frames for each cluster
reference_frames = latent_1_alignment.groupby('cluster')['reference_frame'].first().to_dict()

# Sort the summary DataFrame by cluster number
sorted_summary_df = latent_1_statistics.sort_values(by='Cluster')

# Calculate statistics for each cluster in order of cluster number with a simple progress bar
for cluster_id in tqdm(sorted_summary_df['Cluster'], desc="Processing clusters", unit="cluster"):
    if cluster_id in processed_clusters:
        continue
    indices = cluster_indices[cluster_id]
    reference_idx = reference_frames[cluster_id]
    cluster_stats = calculate_cluster_statistics(cluster_id, indices, reference_idx)
    
    # Convert the dictionary to a DataFrame and concatenate it with the existing DataFrame
    cluster_stats_df = pd.DataFrame([cluster_stats])
    existing_stats_df = pd.concat([existing_stats_df, cluster_stats_df], ignore_index=True)
    existing_stats_df.to_csv(expanded_csv_path, index=False)

print("Processing complete. Cluster statistics have been updated.")

# Generate Figures:

In [ ]:
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import MDAnalysis as mda
import pandas as pd
from MDAnalysis.analysis import align

threshold_val = 1.0
length_val = 1
minimum_indices = 1

# Define the output directories
output_dir = f"N4_Threshold_{threshold_val}_Length_{length_val}"

# Load the CSV files
latent_1_alignment = pd.read_csv(os.path.join(output_dir, "Latent_Alignment.csv"))
latent_1_statistics = pd.read_csv(os.path.join(output_dir, "Cluster_Statistics.csv"))

# Load the MD simulation data
u1 = mda.Universe("myc_091-160_stripped.prmtop", "aMD_091-160_no1-4_stripped.nc")
u2 = mda.Universe("myc_091-160_D132-H_stripped.prmtop", "aMD_091-160_D132-H_no1-4_stripped.nc")

# Define the function to extract frames from both universes
def extract_frames(indices, universe1, universe2, switch_index=40000):
    frames = []
    for idx in indices:
        if idx < switch_index:
            universe1.trajectory[idx]
            frames.append(universe1.select_atoms("name CA").positions.copy())
        else:
            universe2.trajectory[idx - switch_index]
            frames.append(universe2.select_atoms("name CA").positions.copy())
    return frames

# Create a temporary Universe for alignment
def create_temp_universe(positions, universe):
    temp_universe = mda.Merge(universe.select_atoms("name CA"))
    temp_universe.atoms.positions = positions
    return temp_universe

# Align frames using AlignTraj
def align_frames(reference_universe, frames, select_for_alignment):
    aligned_frames = []
    for frame_positions in frames:
        temp_universe = create_temp_universe(frame_positions, reference_universe)
        aligner = align.AlignTraj(temp_universe, reference_universe, select=select_for_alignment, in_memory=False)
        aligner.run()
        aligned_frames.append(temp_universe.atoms.positions.copy())
    return aligned_frames

# Selection string for residues 15-30 to ensure alignment of corresponding residues
select_residues_15_30 = "resid 15-30 and name CA"

# Function to plot aligned frames for a given cluster and save the figures
def plot_aligned_frames(cluster_id, indices, best_reference_idx, save_dir):
    # Define the filename
    filename = f"Cluster_{cluster_id}_N_{len(indices)}.png"
    filepath = os.path.join(save_dir, filename)
    
    # Check if the file already exists
    if (os.path.exists(filepath)):
        print(f"Skipping cluster {cluster_id}, file already exists.")
        return
    
    # Extract frames for the cluster
    frames = extract_frames(indices, u1, u2)
    
    # Find the position of best_reference_idx within the cluster's indices
    reference_idx_in_cluster = indices.index(best_reference_idx)
    
    # Create reference universe for alignment
    reference_universe = create_temp_universe(frames[reference_idx_in_cluster], u1)
    
    # Align frames using the specific residues 15-30
    aligned_frames = align_frames(reference_universe, frames, select_residues_15_30)
    
    # Set the opacity value with a minimum threshold
    opacity = max(1 / len(aligned_frames), 0.002)
    
    # Plot the aligned frames from different angles
    angles = [(0, 0), (90, 0), (0, 90), (90, 90)]
    
    fig, axs = plt.subplots(2, 2, subplot_kw={'projection': '3d'}, figsize=(15, 15))
    
    for ax, angle in zip(axs.flatten(), angles):
        # Plot cluster frames with lines connecting the alpha carbons
        for positions in aligned_frames:
            for i in range(len(positions) - 1):
                if i == 14:
                    color = 'red'  # alpha carbon 105
                elif i == 29:
                    color = 'green'  # alpha carbon 120
                else:
                    color = 'grey'
                ax.plot(positions[i:i+2, 0], positions[i:i+2, 1], positions[i:i+2, 2], 
                        'o-', color=color, markersize=1, markeredgewidth=0.8, alpha=opacity)
        
        ax.view_init(elev=angle[0], azim=angle[1])
        ax.set_axis_off() 
        ax.grid(False)
    
    # Save the figure to the specified directory with the appropriate filename
    plt.savefig(filepath, bbox_inches='tight', pad_inches=0, dpi=300)
    plt.close(fig)

# Create the directory to save the figures if it doesn't exist
save_dir = os.path.join(output_dir, f"Cluster_N4_Full_Threshold_{threshold_val}_All_Angles")
os.makedirs(save_dir, exist_ok=True)

# Group the DataFrame by 'cluster' and get the indices for each cluster
cluster_indices = latent_1_alignment.groupby('cluster').apply(lambda df: df['index'].tolist()).to_dict()

# Sort the summary DataFrame by cluster number
sorted_summary_df = latent_1_statistics.sort_values(by='Cluster')

# Plot aligned frames for each cluster in order of cluster number with a simple progress bar
for cluster_id in tqdm(sorted_summary_df['Cluster'], desc="Processing clusters", unit="cluster"):
    indices = cluster_indices[cluster_id]
    best_reference_idx = latent_1_alignment.loc[latent_1_alignment['cluster'] == cluster_id, 'reference_frame'].values[0]
    plot_aligned_frames(cluster_id, indices, best_reference_idx, save_dir)

# Display Figures:

In [ ]:
# Load the CSV files
threshold_val = 1.0
length_val = 1
minimum_indices = 1

cluster_stats_path = f"N4_Threshold_{threshold_val}_Length_{length_val}/Cluster_Statistics.csv"
expanded_stats_path = f"N4_Threshold_{threshold_val}_Length_{length_val}/Cluster_Statistics_Expanded.csv"
latent_alignment_path = f"N4_Threshold_{threshold_val}_Length_{length_val}/Latent_Alignment.csv"

cluster_stats_df = pd.read_csv(cluster_stats_path)
expanded_stats_df = pd.read_csv(expanded_stats_path)
latent_alignment_df = pd.read_csv(latent_alignment_path)

# Filter clusters with at least 20 frames
filtered_clusters_df = cluster_stats_df[cluster_stats_df['Tot_Dur'] >= 20]

# Merge the filtered clusters with the expanded statistics
merged_df = filtered_clusters_df.merge(expanded_stats_df, on='Cluster')

# Calculate the average values for distance measurements
mean_std_rmsd_avg = merged_df['Std_RMSD'].mean()
mean_dist_15_22_avg = merged_df['Mean_Dist_15_22'].mean()
mean_dist_22_30_avg = merged_df['Mean_Dist_22_30'].mean()
mean_dist_15_30_avg = merged_df['Mean_Dist_15_30'].mean()

# Define the scoring function
def calculate_score(row, weights):
    score = weights['RMSD'] * row['Mean_RMSD']
    score += weights['Std_RMSD_w'] * (row['Std_RMSD'] - mean_std_rmsd_avg)
    score += weights['Dist_15_22'] * (row['Mean_Dist_15_22'] - mean_dist_15_22_avg)
    score += weights['Dist_22_30'] * (row['Mean_Dist_22_30'] - mean_dist_22_30_avg)
    score += weights['Dist_15_30'] * (row['Mean_Dist_15_30'] - mean_dist_15_30_avg)
    return score

# Set weights
weights = {
    'RMSD': 1.0,
    'Std_RMSD_w': 0.5, 
    'Dist_15_22': 0.5,  
    'Dist_22_30': 0.5, 
    'Dist_15_30': 0.5,  
}

# Calculate scores and ranks
tqdm.pandas(desc="Calculating scores")
merged_df['Score'] = merged_df.progress_apply(calculate_score, axis=1, weights=weights)
merged_df['Rank'] = merged_df['Score'].rank(ascending=True)

# Save the results to a CSV file
output_csv_path = f"N4_Threshold_{threshold_val}_Length_{length_val}/Cluster_Rankings.csv"
merged_df.to_csv(output_csv_path, index=False)

# Display the figures
image_folder = f"N4_Threshold_{threshold_val}_Length_{length_val}/Cluster_N4_Full_Threshold_{threshold_val}_All_Angles"

# Sort by rank to display in order
merged_df = merged_df.sort_values(by='Rank', ascending=True)

for _, row in merged_df.iterrows():
    cluster_number = row['Cluster']
    num_frames = row['Tot_Dur']
    rank = row['Rank']
    score = row['Score']
    
    mean_rmsd = row['Mean_RMSD']
    std_rmsd = row['Std_RMSD']
    mean_dist_15_22 = row['Mean_Dist_15_22']
    mean_dist_22_30 = row['Mean_Dist_22_30']
    mean_dist_15_30 = row['Mean_Dist_15_30']
    
    weighted_std_rmsd = weights['Std_RMSD_w'] * (std_rmsd - mean_std_rmsd_avg)
    weighted_dist_15_22 = weights['Dist_15_22'] * (mean_dist_15_22 - mean_dist_15_22_avg)
    weighted_dist_22_30 = weights['Dist_22_30'] * (mean_dist_22_30 - mean_dist_22_30_avg)
    weighted_dist_15_30 = weights['Dist_15_30'] * (mean_dist_15_30 - mean_dist_15_30_avg)
    
    image_path = os.path.join(image_folder, f"Cluster_{int(cluster_number)}_N_{int(num_frames)}.png")
    if os.path.exists(image_path):
        img = plt.imread(image_path)
        plt.figure(figsize=(10, 10))
        plt.imshow(img)
        plt.title(f"Cluster: {cluster_number}, Frames: {num_frames}, Rank: {rank:.2f}, Score: {score:.2f}\n"
                  f"RMSD: {mean_rmsd:.2f}, Weighted Std RMSD: {weighted_std_rmsd:.2f}, "
                  f"Weighted Dist 15-22: {weighted_dist_15_22:.2f}, "
                  f"Weighted Dist 22-30: {weighted_dist_22_30:.2f}, "
                  f"Weighted Dist 15-30: {weighted_dist_15_30:.2f}")
        plt.axis('off')
        plt.show()
    else:
        print(f"Image for Cluster {cluster_number} not found.")